In [ ]:
!pip -q install linearmodels pyreadstat
!pip -q install pyreadstat linearmodels openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # pas une erreur si "already mounted"

from pathlib import Path
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS


#For table 2
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import NegativeBinomial

#For table 6
from statsmodels.stats.sandwich_covariance import cov_cluster
from scipy.stats import chi2


MAIN = Path("/content/drive/MyDrive/replication")
DTA = MAIN / "dta"
FIGURES = MAIN / "figures"
TABLES = MAIN / "tables"

# créer les dossiers si besoin
FIGURES.mkdir(parents=True, exist_ok=True)
TABLES.mkdir(parents=True, exist_ok=True)

print("DTA files:", list(DTA.glob("*"))[:10])


Mounted at /content/drive
DTA files: [PosixPath('/content/drive/MyDrive/replication/dta/replication_file1.dta'), PosixPath('/content/drive/MyDrive/replication/dta/replication_file2.dta'), PosixPath('/content/drive/MyDrive/replication/dta/replication_file3.dta')]


In [ ]:
from pathlib import Path

root = Path("/content/drive/MyDrive")

# Cherche fichiers de données potentiels
patterns = ["*.dta", "*.csv", "*.xlsx", "*.zip", "*.parquet"]
for pat in patterns:
    hits = list(root.rglob(pat))
    print(f"{pat}: {len(hits)}")
    for p in hits[:15]:
        print("  ", p)


*.dta: 3
   /content/drive/MyDrive/replication/dta/replication_file1.dta
   /content/drive/MyDrive/replication/dta/replication_file2.dta
   /content/drive/MyDrive/replication/dta/replication_file3.dta
*.csv: 0
*.xlsx: 17
   /content/drive/MyDrive/DATA_TSPP/epfl_gym_equipment_specifications.xlsx
   /content/drive/MyDrive/DATA_TSPP/epfl_gym_operating_costs.xlsx
   /content/drive/MyDrive/DATA_TSPP/epfl_gym_wait_times_study.xlsx
   /content/drive/MyDrive/DATA_TSPP/epfl_student_demographics.xlsx
   /content/drive/MyDrive/DATA_TSPP/epfl_student_gym_survey.xlsx
   /content/drive/MyDrive/DATA_TSPP/epfl_gym_capacity_data.xlsx
   /content/drive/MyDrive/replication/tables/table_6_a.xlsx
   /content/drive/MyDrive/replication/tables/table_7.xlsx
   /content/drive/MyDrive/replication/tables/table_6_b.xlsx
   /content/drive/MyDrive/replication/tables/table_5c.xlsx
   /content/drive/MyDrive/replication/tables/table_5b.xlsx
   /content/drive/MyDrive/replication/tables/table_5a.xlsx
   /content/drive/My

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


$$
\textbf{Main Tables}
$$


$$ \textbf{TABLE 1 : News Pressure and the Length of Conflict-related News}  $$

In [ ]:

# ---- load data (Stata: use "$dta/replication_file1.dta", clear) ----
df = pd.read_stata(DTA / "replication_file1.dta")

# Helper: run cluster-robust OLS first stage + IV2SLS, then export
def run_table1_panel(df_in: pd.DataFrame, mask: pd.Series, out_xlsx_path, panel_title: str):
    # Variables used in this Stata block
    needed = [
        "daily_woi",
        "daily_woi_nc",
        "length_conflict_news",
        "high_intensity",
        "month",
        "year",
        "dow",
        "monthyear",
    ]

    d = df_in.loc[mask, needed].copy()

    # Stata e(sample): drop missing across all variables used
    d = d.dropna()

    # Fixed effects (Stata: i.month i.year i.dow)
    # statsmodels formula with categorical dummies:
    fe_terms = "C(month) + C(year) + C(dow)"



In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS

# ============================================================
# Table 1 (Panels A & B) — Stata-like layout
# - NO "\n" inside cells
# - Coef and (SE) are on separate rows (like Stata)
# - Stars added from p-values
# ============================================================

def _stars(p):
    if p is None or (not np.isfinite(p)):
        return ""
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

def _dec_adaptive(x, dec=3, dec_small=5, thresh=1e-3):
    try:
        if x is None or (not np.isfinite(float(x))):
            return dec
        return dec_small if abs(float(x)) < thresh else dec
    except Exception:
        return dec

def _fmt_coef(coef, p, dec):
    if coef is None or (not np.isfinite(coef)):
        return ""
    return f"{coef:.{dec}f}{_stars(p)}"

def _fmt_se(se, dec):
    if se is None or (not np.isfinite(se)):
        return ""
    return f"({se:.{dec}f})"

def _get_r2(res):
    # statsmodels OLS/GLM: rsquared; linearmodels IV2SLS: rsquared
    if hasattr(res, "rsquared") and res.rsquared is not None:
        try:
            return float(res.rsquared)
        except Exception:
            return np.nan
    return np.nan

def run_table1_panel(df_in: pd.DataFrame, mask: pd.Series, out_xlsx_path, panel_title: str):
    fe_terms = "C(month) + C(year) + C(dow)"
    base = ["length_conflict_news", "high_intensity", "month", "year", "dow", "monthyear"]

    # -------------------------
    # (A) IV on daily_woi (defines sample like Stata e(sample))
    # -------------------------
    needed_iv1 = ["daily_woi"] + base
    d_iv1 = df_in.loc[mask, needed_iv1].dropna().copy()

    iv1 = IV2SLS.from_formula(
        f"daily_woi ~ 1 + {fe_terms} + [length_conflict_news ~ high_intensity]",
        data=d_iv1
    ).fit(cov_type="clustered", clusters=d_iv1["monthyear"])

    # -------------------------
    # (B) First stage on EXACT same sample as (A)
    # -------------------------
    fs = smf.ols(
        f"length_conflict_news ~ high_intensity + {fe_terms}",
        data=d_iv1
    ).fit(cov_type="cluster", cov_kwds={"groups": d_iv1["monthyear"]})

    ftest = fs.f_test("high_intensity = 0")
    F_excl = float(np.asarray(ftest.fvalue).squeeze()) if hasattr(ftest, "fvalue") else np.nan

    fs_coef = float(fs.params.get("high_intensity", np.nan))
    fs_se   = float(fs.bse.get("high_intensity", np.nan))
    fs_p    = float(fs.pvalues.get("high_intensity", np.nan))
    fs_n    = int(fs.nobs)

    # IV key coef is on length_conflict_news in equation for daily_woi
    iv1_coef = float(iv1.params.get("length_conflict_news", np.nan))
    iv1_se   = float(iv1.std_errors.get("length_conflict_news", np.nan))
    iv1_p    = float(iv1.pvalues.get("length_conflict_news", np.nan))
    iv1_n    = int(iv1.nobs)

    # -------------------------
    # (C) IV on daily_woi_nc (can be different sample)
    # -------------------------
    needed_iv2 = ["daily_woi_nc"] + base
    d_iv2 = df_in.loc[mask, needed_iv2].dropna().copy()

    iv2 = IV2SLS.from_formula(
        f"daily_woi_nc ~ 1 + {fe_terms} + [length_conflict_news ~ high_intensity]",
        data=d_iv2
    ).fit(cov_type="clustered", clusters=d_iv2["monthyear"])

    iv2_coef = float(iv2.params.get("length_conflict_news", np.nan))
    iv2_se   = float(iv2.std_errors.get("length_conflict_news", np.nan))
    iv2_p    = float(iv2.pvalues.get("length_conflict_news", np.nan))
    iv2_n    = int(iv2.nobs)

    # -------------------------
    # Stata-like table: coef row + SE row (NO "\n" in cells)
    # -------------------------
    col1 = "Length conflict news, 1st stage"
    col2 = "NP, 2SLS"
    col3 = "Uncorr NP, 2SLS"
    cols = [col1, col2, col3]

    # adaptive decimals (keeps tiny coef readable like 0.00001)
    d_fs  = _dec_adaptive(fs_coef)
    d_iv1 = _dec_adaptive(iv1_coef)
    d_iv2 = _dec_adaptive(iv2_coef)

    # rows (exactly like your Stata screenshot)
    r1 = "Periods of high intensity of conflict"
    r2 = "Length of conflict-related news stories (in minutes)"
    # blank labels for SE lines (so it visually matches Stata)
    r1_se = ""
    r2_se = " "

    table = pd.DataFrame("", index=[r1, r1_se, r2, r2_se], columns=cols)

    # First stage: show high_intensity
    table.loc[r1,    col1] = _fmt_coef(fs_coef, fs_p, d_fs)
    table.loc[r1_se, col1] = _fmt_se(fs_se, d_fs)

    # 2SLS: show length_conflict_news
    table.loc[r2,    col2] = _fmt_coef(iv1_coef, iv1_p, d_iv1)
    table.loc[r2_se, col2] = _fmt_se(iv1_se, d_iv1)

    table.loc[r2,    col3] = _fmt_coef(iv2_coef, iv2_p, d_iv2)
    table.loc[r2_se, col3] = _fmt_se(iv2_se, d_iv2)

    # -------------------------
    # Stats block (like Stata)
    # -------------------------
    stats = pd.DataFrame("", index=["Observations", "R-squared", "F excl. instr."], columns=cols)
    stats.loc["Observations", col1] = fs_n
    stats.loc["Observations", col2] = iv1_n
    stats.loc["Observations", col3] = iv2_n

    r2_fs  = _get_r2(fs)
    r2_iv1 = _get_r2(iv1)
    r2_iv2 = _get_r2(iv2)
    stats.loc["R-squared", col1] = "" if not np.isfinite(r2_fs)  else round(r2_fs, 3)
    stats.loc["R-squared", col2] = "" if not np.isfinite(r2_iv1) else round(r2_iv1, 3)
    stats.loc["R-squared", col3] = "" if not np.isfinite(r2_iv2) else round(r2_iv2, 3)

    stats.loc["F excl. instr.", col1] = round(F_excl, 2) if np.isfinite(F_excl) else ""
    stats.loc["F excl. instr.", col2] = round(F_excl, 2) if np.isfinite(F_excl) else ""
    stats.loc["F excl. instr.", col3] = round(F_excl, 2) if np.isfinite(F_excl) else ""

    # Footnotes like Stata (optional, but matches your screenshot)
    notes = pd.DataFrame(
        {"": ["Robust standard errors in parentheses", "*** p<0.01, ** p<0.05, * p<0.1"]},
        index=["", " "]
    )

    # -------------------------
    # Export
    # -------------------------
    out_xlsx_path = str(out_xlsx_path)
    with pd.ExcelWriter(out_xlsx_path, engine="openpyxl") as writer:
        title_df = pd.DataFrame(
            {"": [
                "*** MAIN TABLES ***",
                "Table 1. News Pressure and the Length of Conflict-related News",
                panel_title,
                ""
            ]}
        )
        title_df.to_excel(writer, sheet_name="Table 1", index=False, header=False, startrow=0)
        table.to_excel(writer, sheet_name="Table 1", startrow=6)
        stats.to_excel(writer, sheet_name="Table 1", startrow=6 + len(table) + 2)
        notes.to_excel(writer, sheet_name="Table 1", index=False, header=False, startrow=6 + len(table) + 2 + len(stats) + 2)

    print(f"Saved: {out_xlsx_path}")
    display(table)
    display(stats)
    return table, stats


# ============================================================
# RUN PANELS A & B (same output structure as Stata)
# ============================================================

# Load once
df = pd.read_stata(DTA / "replication_file1.dta").copy()

# Panel A: Full sample
mask_full = pd.Series(True, index=df.index)
run_table1_panel(
    df_in=df,
    mask=mask_full,
    out_xlsx_path=TABLES / "table_1a.xlsx",
    panel_title="Panel A: Full sample"
)

# Panel B: Attack on same day or previous day
mask_attack = (df["occurrence_t_y"] == 1) | (df["occurrence_pal_t_y"] == 1)
run_table1_panel(
    df_in=df,
    mask=mask_attack,
    out_xlsx_path=TABLES / "table_1b.xlsx",
    panel_title="Panel B: Attack on same day or previous day"
)


Saved: /content/drive/MyDrive/replication/tables/table_1a.xlsx


,"Length conflict news, 1st stage","NP, 2SLS","Uncorr NP, 2SLS"
Periods of high intensity of conflict,5.046***,,
,(1.330),,
Length of conflict-related news stories (in minutes),,0.002,-0.017***
,,(0.006),(0.005)


,"Length conflict news, 1st stage","NP, 2SLS","Uncorr NP, 2SLS"
Observations,4003,4003,4003
R-squared,0.288,0.096,0.144
F excl. instr.,14.4,14.4,14.4


Saved: /content/drive/MyDrive/replication/tables/table_1b.xlsx


,"Length conflict news, 1st stage","NP, 2SLS","Uncorr NP, 2SLS"
Periods of high intensity of conflict,5.291***,,
,(1.235),,
Length of conflict-related news stories (in minutes),,0.00001,-0.018***
,,(0.00603),(0.005)


,"Length conflict news, 1st stage","NP, 2SLS","Uncorr NP, 2SLS"
Observations,2331,2331,2331
R-squared,0.295,0.133,0.2
F excl. instr.,18.35,18.35,18.35


(                                                   Length conflict news, 1st stage  \
 Periods of high intensity of conflict                                     5.291***   
                                                                            (1.235)   
 Length of conflict-related news stories (in min...                                   
                                                                                      
 
                                                      NP, 2SLS Uncorr NP, 2SLS  
 Periods of high intensity of conflict                                          
                                                                                
 Length of conflict-related news stories (in min...    0.00001       -0.018***  
                                                     (0.00603)         (0.005)  ,
                Length conflict news, 1st stage NP, 2SLS Uncorr NP, 2SLS
 Observations                              2331     2331            2331
 R-squared 

$$ \textbf{TABLE 2 : Coverage of Conflict, News Pressure, and Google Searches}  $$

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display

# ---- load data (Stata: use "$dta/replication_file1.dta", clear) ----
df = pd.read_stata(DTA / "replication_file1.dta")

def run_table2(df_in: pd.DataFrame, out_xlsx_path):
    fe_terms = "C(month) + C(year) + C(dow)"

    # -------------------------
    # Helpers: stars + safe extract
    # -------------------------
    def stars(p):
        if p < 0.01:
            return "***"
        elif p < 0.05:
            return "**"
        elif p < 0.10:
            return "*"
        return ""

    def get_param(res, var):
        if (not hasattr(res, "params")) or (not hasattr(res.params, "index")) or (var not in res.params.index):
            return np.nan, np.nan, np.nan
        b = float(res.params[var])
        se = float(res.bse[var]) if (hasattr(res, "bse") and var in res.bse.index) else np.nan
        p = float(res.pvalues[var]) if (hasattr(res, "pvalues") and var in res.pvalues.index) else np.nan
        return b, se, p

    def fmt_b_se(res, var, dec=3):
        b, se, p = get_param(res, var)
        if not np.isfinite(b) or not np.isfinite(se):
            return "", ""
        st = "" if (not np.isfinite(p)) else stars(p)
        b_txt = f"{b:.{dec}f}{st}"
        se_txt = f"({se:.{dec}f})"
        return b_txt, se_txt

    def get_nobs(res):
        if hasattr(res, "nobs"):
            try:
                return int(res.nobs)
            except Exception:
                pass
        if hasattr(res, "model") and hasattr(res.model, "endog"):
            return int(len(res.model.endog))
        return np.nan

    def get_r2(res):
        # OLS has rsquared; GLM generally doesn't (leave blank)
        if hasattr(res, "rsquared"):
            try:
                return float(res.rsquared)
            except Exception:
                return ""
        return ""

    # -------------------------
    # Fit models (same structure as your current Table 2)
    # -------------------------

    # Col (1): OLS, cluster(monthyear)
    needed1 = ["any_conflict_news", "occurrence_t_y", "occurrence_pal_t_y", "month", "year", "dow", "monthyear"]
    d1 = df_in[needed1].dropna().copy()
    m1 = smf.ols(
        f"any_conflict_news ~ occurrence_t_y + occurrence_pal_t_y + {fe_terms}",
        data=d1
    ).fit(cov_type="cluster", cov_kwds={"groups": d1["monthyear"]})

    # Col (2): NB, cluster(monthyear)
    needed2 = ["length_conflict_news", "occurrence_t_y", "occurrence_pal_t_y", "month", "year", "dow", "monthyear"]
    d2 = df_in[needed2].dropna().copy()
    m2 = smf.glm(
        f"length_conflict_news ~ occurrence_t_y + occurrence_pal_t_y + {fe_terms}",
        data=d2,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": d2["monthyear"]})

    # Attack sample for cols (3)-(4)
    mask_attack = (df_in["occurrence_t_y"] == 1) | (df_in["occurrence_pal_t_y"] == 1)

    # Col (3): OLS, cluster(monthyear)
    needed3 = ["any_conflict_news", "lnvic_t_y", "lnvic_pal_y", "daily_woi", "month", "year", "dow", "monthyear"]
    d3 = df_in.loc[mask_attack, needed3].dropna().copy()
    m3 = smf.ols(
        f"any_conflict_news ~ lnvic_t_y + lnvic_pal_y + daily_woi + {fe_terms}",
        data=d3
    ).fit(cov_type="cluster", cov_kwds={"groups": d3["monthyear"]})

    # Col (4): NB, cluster(monthyear)
    needed4 = ["length_conflict_news", "lnvic_t_y", "lnvic_pal_y", "daily_woi", "month", "year", "dow", "monthyear"]
    d4 = df_in.loc[mask_attack, needed4].dropna().copy()
    m4 = smf.glm(
        f"length_conflict_news ~ lnvic_t_y + lnvic_pal_y + daily_woi + {fe_terms}",
        data=d4,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": d4["monthyear"]})

    # Newey-West (HAC) ordering needs time sort
    needed5 = ["conflict_searches", "lnvic_t_y", "lnvic_pal_y", "monthyear", "month", "year", "dow", "length_conflict_news_t_t_1"]
    if "date" in df_in.columns:
        needed5 = needed5 + ["date"]

    # Col (5): OLS HAC(7) (no lag length in rhs)
    d5 = df_in[needed5].dropna().copy()
    if "date" in d5.columns:
        d5 = d5.sort_values("date")
    m5 = smf.ols(
        f"conflict_searches ~ lnvic_t_y + lnvic_pal_y + monthyear + {fe_terms}",
        data=d5
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # Col (6): OLS HAC(7) + lag length in rhs
    d6 = df_in[needed5].dropna().copy()
    if "date" in d6.columns:
        d6 = d6.sort_values("date")
    m6 = smf.ols(
        f"conflict_searches ~ lnvic_t_y + lnvic_pal_y + length_conflict_news_t_t_1 + monthyear + {fe_terms}",
        data=d6
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # If you really have (7) and (8) in your Stata table, plug them here.
    # For now, keep placeholders (you can replace m7/m8 with your actual fitted models).
    m7 = None
    m8 = None

    # -------------------------
    # Column headers exactly like the table (you can tweak wording)
    # -------------------------
    col_headers = {
        "(1)": "Isr-Pal on news",
        "(2)": "Time to Isr-Pal news",
        "(3)": "Time to Isr-Pal news",
        "(4)": "Isr-Pal on news",
        "(5)": "Time to Isr-Pal news",
        "(6)": "Time to Isr-Pal news",
        "(7)": "Google searches for Israeli-Palestinian conflict",
        "(8)": "Google searches for Israeli-Palestinian conflict",
    }

    models = {
        "(1)": (m1, ["occurrence_t_y", "occurrence_pal_t_y"]),
        "(2)": (m2, ["occurrence_t_y", "occurrence_pal_t_y"]),
        "(3)": (m3, ["lnvic_t_y", "lnvic_pal_y", "daily_woi"]),
        "(4)": (m4, ["lnvic_t_y", "lnvic_pal_y", "daily_woi"]),
        "(5)": (m5, ["lnvic_t_y", "lnvic_pal_y"]),
        "(6)": (m6, ["lnvic_t_y", "lnvic_pal_y", "length_conflict_news_t_t_1"]),
        "(7)": (m7, []),  # replace with your real model + keep list
        "(8)": (m8, []),  # replace with your real model + keep list
    }

    # -------------------------
    # Variable labels exactly like your Excel screenshot
    # (each var becomes 2 rows: coef row + SE row)
    # -------------------------
    var_blocks = [
        ("lnvic_t_y", "Ln (+1) of victims of Israeli attacks (same and previous day)"),
        ("lnvic_pal_y", "Ln (+1) of victims of Palestinian attacks (same and previous day)"),
        ("daily_woi", "News pressure"),
        ("occurrence_t_y", "Occurrence of Israeli attacks (same or previous day)"),
        ("occurrence_pal_t_y", "Occurrence of Palestinian attacks (same or previous day)"),
        ("length_conflict_news_t_t_1", "Length of conflict-related news stories (same and previous day)"),
    ]

    # -------------------------
    # Build a Stata-style table: coef and SE on separate rows (no '\n' in cells)
    # -------------------------
    out_cols = ["VARIABLES"] + [f"{k} {col_headers[k]}" for k in models.keys()]

    rows = []
    for _, label in var_blocks:
        rows.append(label)   # coef row label
        rows.append("")      # SE row label

    out = pd.DataFrame("", index=range(len(rows)), columns=out_cols)
    out["VARIABLES"] = rows

    # Fill cells
    r = 0
    for var, label in var_blocks:
        for ck, (res, keep_vars) in models.items():
            colname = f"{ck} {col_headers[ck]}"
            if (res is None) or (var not in keep_vars):
                out.loc[r, colname] = ""
                out.loc[r + 1, colname] = ""
            else:
                b_txt, se_txt = fmt_b_se(res, var, dec=3)
                out.loc[r, colname] = b_txt
                out.loc[r + 1, colname] = se_txt
        r += 2

    # -------------------------
    # Add stats rows like your screenshot
    # -------------------------
    stats_start = len(out)
    stats_rows = [
        ("Observations", {k: get_nobs(models[k][0]) if models[k][0] is not None else "" for k in models}),
        ("R-squared",    {k: get_r2(models[k][0]) if models[k][0] is not None else "" for k in models}),
        ("Robust standard errors in parentheses", {k: "" for k in models}),
        ("*** p<0.01, ** p<0.05, * p<0.1", {k: "" for k in models}),
    ]

    add = pd.DataFrame("", index=range(len(stats_rows)), columns=out_cols)
    for i, (lab, vals) in enumerate(stats_rows):
        add.loc[i, "VARIABLES"] = lab
        for ck in models:
            add.loc[i, f"{ck} {col_headers[ck]}"] = vals.get(ck, "")

    out_final = pd.concat([out, add], axis=0, ignore_index=True)

    # -------------------------
    # Export + display
    # -------------------------
    out_xlsx_path = str(out_xlsx_path)
    with pd.ExcelWriter(out_xlsx_path, engine="openpyxl") as writer:
        title_df = pd.DataFrame({"": [
            "*** MAIN TABLES ***",
            "Table 2. Coverage of Conflict, News Pressure, and Google Searches",
            "",
            ""
        ]})
        title_df.to_excel(writer, sheet_name="Table 2", index=False, header=False, startrow=0)
        out_final.to_excel(writer, sheet_name="Table 2", index=False, startrow=4)

    print(f"Saved: {out_xlsx_path}")
    display(out_final)

    return out_final

# Run
table2 = run_table2(df, TABLES / "table_2.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_2.xlsx


,VARIABLES,(1) Isr-Pal on news,(2) Time to Isr-Pal news,(3) Time to Isr-Pal news,(4) Isr-Pal on news,(5) Time to Isr-Pal news,(6) Time to Isr-Pal news,(7) Google searches for Israeli-Palestinian conflict,(8) Google searches for Israeli-Palestinian conflict
0,Ln (+1) of victims of Israeli attacks (same an...,,,0.169***,0.765***,0.130***,0.053*,,
1,,,,(0.016),(0.083),(0.043),(0.028),,
2,Ln (+1) of victims of Palestinian attacks (sam...,,,0.134***,0.527***,0.042,0.005,,
3,,,,(0.024),(0.086),(0.059),(0.054),,
4,News pressure,,,-0.078*,-0.676***,,,,
5,,,,(0.042),(0.259),,,,
6,Occurrence of Israeli attacks (same or previou...,0.100***,1.011***,,,,,,
7,,(0.020),(0.201),,,,,,
8,Occurrence of Palestinian attacks (same or pre...,0.112***,0.666***,,,,,,
9,,(0.032),(0.119),,,,,,


$$ \textbf{Table 3 : Israeli Attacks and News Pressure}$$

In [ ]:
# Liste toutes les colonnes qui contiennent "lagdaily" / "leaddaily"
lag_cols = sorted([c for c in df.columns if "lagdaily" in c])
lead_cols = sorted([c for c in df.columns if "leaddaily" in c])

print("Lag columns found:", lag_cols[:50])
print("Lead columns found:", lead_cols[:50])


Lag columns found: ['lagdaily_woi', 'lagdaily_woi2', 'lagdaily_woi2_c_med', 'lagdaily_woi2_long', 'lagdaily_woi2_nc', 'lagdaily_woi3', 'lagdaily_woi3_c_med', 'lagdaily_woi3_long', 'lagdaily_woi3_nc', 'lagdaily_woi4', 'lagdaily_woi4_c_med', 'lagdaily_woi4_long', 'lagdaily_woi4_nc', 'lagdaily_woi5', 'lagdaily_woi5_c_med', 'lagdaily_woi5_long', 'lagdaily_woi5_nc', 'lagdaily_woi6', 'lagdaily_woi6_c_med', 'lagdaily_woi6_long', 'lagdaily_woi6_nc', 'lagdaily_woi7', 'lagdaily_woi7_c_med', 'lagdaily_woi7_long', 'lagdaily_woi7_nc', 'lagdaily_woi_c_med', 'lagdaily_woi_long', 'lagdaily_woi_nc']
Lead columns found: ['leaddaily_woi', 'leaddaily_woi2', 'leaddaily_woi2_nc', 'leaddaily_woi3', 'leaddaily_woi3_nc', 'leaddaily_woi4', 'leaddaily_woi4_nc', 'leaddaily_woi5', 'leaddaily_woi5_nc', 'leaddaily_woi6', 'leaddaily_woi6_nc', 'leaddaily_woi7', 'leaddaily_woi7_nc', 'leaddaily_woi_c_med', 'leaddaily_woi_long', 'leaddaily_woi_nc']


In [ ]:
# ---- load data (Stata: use "$dta/replication_file1.dta", clear) ----
df = pd.read_stata(DTA / "replication_file1.dta").copy()

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display

def run_table3(df_in: pd.DataFrame, out_xlsx_path):
    # Stata: sort date (important for Newey-West HAC ordering)
    d0 = df_in.copy()
    if "date" in d0.columns:
        d0 = d0.sort_values("date")

    fe_terms = "C(month) + C(year) + C(dow)"

    # =========================
    # Helpers: stars + Stata-like formatting (SINGLE LINE)
    # =========================
    def stars(p):
        if p < 0.01:
            return "***"
        elif p < 0.05:
            return "**"
        elif p < 0.10:
            return "*"
        else:
            return ""

    def fmt_stata(res, var, dec=3):
        """
        SINGLE-LINE cell: 'coef*** (se)' so no '\n' appears.
        """
        if (not hasattr(res, "params")) or (not hasattr(res.params, "index")) or (var not in res.params.index):
            return ""

        coef = float(res.params[var])

        # robust extraction of se and p
        try:
            se = float(res.bse[var])
        except Exception:
            se = np.nan

        try:
            p = float(res.pvalues[var])
        except Exception:
            p = np.nan

        st = "" if (not np.isfinite(p)) else stars(p)

        if not np.isfinite(se):
            return f"{coef:.{dec}f}{st} (.)"
        return f"{coef:.{dec}f}{st} ({se:.{dec}f})"

    # =========================
    # PANEL A: News Pressure (daily_woi) — Stata: if gaza_war==0
    # =========================
    dA = d0[d0["gaza_war"] == 0].copy()

    # --- Spec 1: reg occurrence daily_woi ... vce(cluster monthyear)
    needed1 = ["occurrence", "daily_woi", "month", "year", "dow", "monthyear"]
    s1 = dA[needed1].dropna().copy()
    m1 = smf.ols(f"occurrence ~ daily_woi + {fe_terms}", data=s1).fit(
        cov_type="cluster", cov_kwds={"groups": s1["monthyear"]}
    )

    # --- Spec 2: newey occurrence daily_woi leaddaily_woi ... lag(7)
    needed2 = ["occurrence", "daily_woi", "leaddaily_woi", "month", "year", "dow", "monthyear"]
    s2 = dA[needed2].dropna().copy()
    if "date" in s2.columns:
        s2 = s2.sort_values("date")
    m2 = smf.ols(f"occurrence ~ daily_woi + leaddaily_woi + {fe_terms}", data=s2).fit(
        cov_type="HAC", cov_kwds={"maxlags": 7}
    )

    # --- Spec 3: newey occurrence daily_woi leaddaily_woi lags + pal controls ... lag(7)
    needed3 = [
        "occurrence", "daily_woi", "leaddaily_woi",
        "lagdaily_woi", "lagdaily_woi2", "lagdaily_woi3", "lagdaily_woi4", "lagdaily_woi5", "lagdaily_woi6", "lagdaily_woi7",
        "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
        "month", "year", "dow", "monthyear"
    ]
    s3 = dA[needed3].dropna().copy()
    if "date" in s3.columns:
        s3 = s3.sort_values("date")

    rhs_lags = "lagdaily_woi + lagdaily_woi2 + lagdaily_woi3 + lagdaily_woi4 + lagdaily_woi5 + lagdaily_woi6 + lagdaily_woi7"
    rhs_pal  = "occurrence_pal_1 + occurrence_pal_2_7 + occurrence_pal_8_14"

    m3 = smf.ols(
        f"occurrence ~ daily_woi + leaddaily_woi + {rhs_lags} + {rhs_pal} + {fe_terms}",
        data=s3
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # --- Spec 4: reg lnvic daily_woi ... cluster(monthyear)
    needed4 = ["lnvic", "daily_woi", "month", "year", "dow", "monthyear"]
    s4 = dA[needed4].dropna().copy()
    m4 = smf.ols(f"lnvic ~ daily_woi + {fe_terms}", data=s4).fit(
        cov_type="cluster", cov_kwds={"groups": s4["monthyear"]}
    )

    # --- Spec 5: newey lnvic daily_woi leaddaily_woi ... lag(7)
    needed5 = ["lnvic", "daily_woi", "leaddaily_woi", "month", "year", "dow", "monthyear"]
    s5 = dA[needed5].dropna().copy()
    if "date" in s5.columns:
        s5 = s5.sort_values("date")
    m5 = smf.ols(f"lnvic ~ daily_woi + leaddaily_woi + {fe_terms}", data=s5).fit(
        cov_type="HAC", cov_kwds={"maxlags": 7}
    )

    # --- Spec 6: newey lnvic daily_woi leaddaily_woi lags + pal controls ... lag(7)
    needed6 = [
        "lnvic", "daily_woi", "leaddaily_woi",
        "lagdaily_woi", "lagdaily_woi2", "lagdaily_woi3", "lagdaily_woi4", "lagdaily_woi5", "lagdaily_woi6", "lagdaily_woi7",
        "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
        "month", "year", "dow", "monthyear"
    ]
    s6 = dA[needed6].dropna().copy()
    if "date" in s6.columns:
        s6 = s6.sort_values("date")

    m6 = smf.ols(
        f"lnvic ~ daily_woi + leaddaily_woi + {rhs_lags} + {rhs_pal} + {fe_terms}",
        data=s6
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # --- Spec 7: glm victims_isr ... family(nbinom) vce(hac nwest 7)
    needed7 = [
        "victims_isr", "daily_woi", "leaddaily_woi",
        "lagdaily_woi", "lagdaily_woi2", "lagdaily_woi3", "lagdaily_woi4", "lagdaily_woi5", "lagdaily_woi6", "lagdaily_woi7",
        "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
        "month", "year", "dow"
    ]
    if "date" in dA.columns:
        needed7 = needed7 + ["date"]

    s7 = dA[needed7].dropna().copy()
    if "date" in s7.columns:
        s7 = s7.sort_values("date")

    m7 = smf.glm(
        f"victims_isr ~ daily_woi + leaddaily_woi + {rhs_lags} + {rhs_pal} + {fe_terms}",
        data=s7,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # =========================
    # PANEL B: Uncorrected News Pressure (daily_woi_nc)
    # =========================
    dB = d0[d0["gaza_war"] == 0].copy()

    # Spec 1B
    needed1b = ["occurrence", "daily_woi_nc", "month", "year", "dow", "monthyear"]
    t1b = dB[needed1b].dropna().copy()
    b1 = smf.ols(f"occurrence ~ daily_woi_nc + {fe_terms}", data=t1b).fit(
        cov_type="cluster", cov_kwds={"groups": t1b["monthyear"]}
    )

    # Spec 2B
    needed2b = ["occurrence", "daily_woi_nc", "leaddaily_woi_nc", "month", "year", "dow", "monthyear"]
    t2b = dB[needed2b].dropna().copy()
    if "date" in t2b.columns:
        t2b = t2b.sort_values("date")
    b2 = smf.ols(f"occurrence ~ daily_woi_nc + leaddaily_woi_nc + {fe_terms}", data=t2b).fit(
        cov_type="HAC", cov_kwds={"maxlags": 7}
    )

    # Spec 3B
    needed3b = [
        "occurrence", "daily_woi_nc", "leaddaily_woi_nc",
        "lagdaily_woi_nc", "lagdaily_woi2_nc", "lagdaily_woi3_nc", "lagdaily_woi4_nc", "lagdaily_woi5_nc", "lagdaily_woi6_nc", "lagdaily_woi7_nc",
        "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
        "month", "year", "dow", "monthyear"
    ]
    t3b = dB[needed3b].dropna().copy()
    if "date" in t3b.columns:
        t3b = t3b.sort_values("date")

    rhs_lags_nc = "lagdaily_woi_nc + lagdaily_woi2_nc + lagdaily_woi3_nc + lagdaily_woi4_nc + lagdaily_woi5_nc + lagdaily_woi6_nc + lagdaily_woi7_nc"

    b3 = smf.ols(
        f"occurrence ~ daily_woi_nc + leaddaily_woi_nc + {rhs_lags_nc} + {rhs_pal} + {fe_terms}",
        data=t3b
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # lnvic with nc
    needed4b = ["lnvic", "daily_woi_nc", "month", "year", "dow", "monthyear"]
    t4b = dB[needed4b].dropna().copy()
    b4 = smf.ols(f"lnvic ~ daily_woi_nc + {fe_terms}", data=t4b).fit(
        cov_type="cluster", cov_kwds={"groups": t4b["monthyear"]}
    )

    needed5b = ["lnvic", "daily_woi_nc", "leaddaily_woi_nc", "month", "year", "dow", "monthyear"]
    t5b = dB[needed5b].dropna().copy()
    if "date" in t5b.columns:
        t5b = t5b.sort_values("date")
    b5 = smf.ols(f"lnvic ~ daily_woi_nc + leaddaily_woi_nc + {fe_terms}", data=t5b).fit(
        cov_type="HAC", cov_kwds={"maxlags": 7}
    )

    needed6b = [
        "lnvic", "daily_woi_nc", "leaddaily_woi_nc",
        "lagdaily_woi_nc", "lagdaily_woi2_nc", "lagdaily_woi3_nc", "lagdaily_woi4_nc", "lagdaily_woi5_nc", "lagdaily_woi6_nc", "lagdaily_woi7_nc",
        "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
        "month", "year", "dow", "monthyear"
    ]
    t6b = dB[needed6b].dropna().copy()
    if "date" in t6b.columns:
        t6b = t6b.sort_values("date")

    b6 = smf.ols(
        f"lnvic ~ daily_woi_nc + leaddaily_woi_nc + {rhs_lags_nc} + {rhs_pal} + {fe_terms}",
        data=t6b
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # Spec 7B: NB victims with nc + HAC(7)
    needed7b = [
        "victims_isr", "daily_woi_nc", "leaddaily_woi_nc",
        "lagdaily_woi_nc", "lagdaily_woi2_nc", "lagdaily_woi3_nc", "lagdaily_woi4_nc", "lagdaily_woi5_nc", "lagdaily_woi6_nc", "lagdaily_woi7_nc",
        "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
        "month", "year", "dow"
    ]
    if "date" in dB.columns:
        needed7b = needed7b + ["date"]

    t7b = dB[needed7b].dropna().copy()
    if "date" in t7b.columns:
        t7b = t7b.sort_values("date")

    b7 = smf.glm(
        f"victims_isr ~ daily_woi_nc + leaddaily_woi_nc + {rhs_lags_nc} + {rhs_pal} + {fe_terms}",
        data=t7b,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # =========================
    # Build output tables
    # =========================
    panelA_models = [m1, m2, m3, m4, m5, m6, m7]
    panelA_names  = ["Occur (clust)", "Occur (NW)", "Occur (NW+lags)",
                     "Ln(vic) (clust)", "Ln(vic) (NW)", "Ln(vic) (NW+lags)",
                     "Num. victims (NB, NW)"]
    panelA_vars   = ["daily_woi", "leaddaily_woi", "lagdaily_woi",
                     "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14"]

    A = pd.DataFrame(index=panelA_vars, columns=panelA_names)
    for name, res in zip(panelA_names, panelA_models):
        for v in panelA_vars:
            A.loc[v, name] = fmt_stata(res, v, 3)

    A_stats = pd.DataFrame(index=["N"], columns=panelA_names)
    for name, res in zip(panelA_names, panelA_models):
        A_stats.loc["N", name] = int(res.nobs)

    panelB_models = [b1, b2, b3, b4, b5, b6, b7]
    panelB_names  = ["Occur_nc (clust)", "Occur_nc (NW)", "Occur_nc (NW+lags)",
                     "Ln(vic)_nc (clust)", "Ln(vic)_nc (NW)", "Ln(vic)_nc (NW+lags)",
                     "Num. victims (NB, NW)"]
    panelB_vars   = ["daily_woi_nc", "leaddaily_woi_nc", "lagdaily_woi_nc",
                     "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14"]

    B = pd.DataFrame(index=panelB_vars, columns=panelB_names)
    for name, res in zip(panelB_names, panelB_models):
        for v in panelB_vars:
            B.loc[v, name] = fmt_stata(res, v, 3)

    B_stats = pd.DataFrame(index=["N"], columns=panelB_names)
    for name, res in zip(panelB_names, panelB_models):
        B_stats.loc["N", name] = int(res.nobs)

    # Export Excel
    out_xlsx_path = str(out_xlsx_path)
    with pd.ExcelWriter(out_xlsx_path, engine="openpyxl") as writer:
        title_df = pd.DataFrame({"": [
            "*** MAIN TABLES ***",
            "Table 3. Israeli Attacks and News Pressure",
            "",
            ""
        ]})
        title_df.to_excel(writer, sheet_name="Table 3A", index=False, header=False, startrow=0)
        A.to_excel(writer, sheet_name="Table 3A", startrow=6)
        A_stats.to_excel(writer, sheet_name="Table 3A", startrow=6 + len(A) + 3)

        title_df.to_excel(writer, sheet_name="Table 3B", index=False, header=False, startrow=0)
        B.to_excel(writer, sheet_name="Table 3B", startrow=6)
        B_stats.to_excel(writer, sheet_name="Table 3B", startrow=6 + len(B) + 3)

    print(f"Saved: {out_xlsx_path}")
    print("Panel A Ns:", [int(m.nobs) for m in panelA_models])
    print("Panel B Ns:", [int(m.nobs) for m in panelB_models])

    display(A.astype(str).replace("nan", ""))
    display(A_stats.astype(str).replace("nan", ""))

    display(B.astype(str).replace("nan", ""))
    display(B_stats.astype(str).replace("nan", ""))

    return A, A_stats, B, B_stats

# Run Table 3
A, A_stats, B, B_stats = run_table3(df, TABLES / "table_3.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_3.xlsx
Panel A Ns: [4048, 4045, 4024, 4048, 4045, 4024, 4024]
Panel B Ns: [4048, 4045, 4024, 4048, 4045, 4024, 4024]


,Occur (clust),Occur (NW),Occur (NW+lags),Ln(vic) (clust),Ln(vic) (NW),Ln(vic) (NW+lags),"Num. victims (NB, NW)"
daily_woi,0.074** (0.032),0.030 (0.034),0.026 (0.035),0.130** (0.052),0.058 (0.050),0.028 (0.046),0.020 (0.136)
leaddaily_woi,,0.084** (0.033),0.078** (0.035),,0.138*** (0.047),0.121** (0.048),0.466*** (0.151)
lagdaily_woi,,,-0.026 (0.034),,,-0.035 (0.045),-0.185 (0.155)
occurrence_pal_1,,,0.104*** (0.030),,,0.220*** (0.057),0.429*** (0.098)
occurrence_pal_2_7,,,0.086*** (0.021),,,0.168*** (0.036),0.402*** (0.090)
occurrence_pal_8_14,,,0.098*** (0.022),,,0.141*** (0.035),0.291*** (0.087)


,Occur (clust),Occur (NW),Occur (NW+lags),Ln(vic) (clust),Ln(vic) (NW),Ln(vic) (NW+lags),"Num. victims (NB, NW)"
N,4048,4045,4024,4048,4045,4024,4024


,Occur_nc (clust),Occur_nc (NW),Occur_nc (NW+lags),Ln(vic)_nc (clust),Ln(vic)_nc (NW),Ln(vic)_nc (NW+lags),"Num. victims (NB, NW)"
daily_woi_nc,0.027 (0.034),-0.007 (0.034),-0.003 (0.036),0.021 (0.065),-0.023 (0.053),-0.037 (0.049),-0.222 (0.150)
leaddaily_woi_nc,,0.063* (0.034),0.063* (0.035),,0.080* (0.048),0.076 (0.048),0.312** (0.154)
lagdaily_woi_nc,,,-0.030 (0.035),,,-0.031 (0.046),-0.122 (0.153)
occurrence_pal_1,,,0.105*** (0.030),,,0.221*** (0.056),0.419*** (0.097)
occurrence_pal_2_7,,,0.086*** (0.021),,,0.168*** (0.036),0.403*** (0.092)
occurrence_pal_8_14,,,0.099*** (0.022),,,0.144*** (0.036),0.297*** (0.088)


,Occur_nc (clust),Occur_nc (NW),Occur_nc (NW+lags),Ln(vic)_nc (clust),Ln(vic)_nc (NW),Ln(vic)_nc (NW+lags),"Num. victims (NB, NW)"
N,4048,4045,4024,4048,4045,4024,4024


$$ \textbf{TABLE 4: Palestinian Attacks and News Pressure} $$

In [ ]:
### new version table 4 (NO "\n" in coef cells)

from IPython.display import display
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

def run_table4(df_in: pd.DataFrame, out_xlsx_path):
    # -------------------------
    # Stata: use ..., sort date, if gaza_war==0
    # -------------------------
    d0 = df_in[df_in["gaza_war"] == 0].copy()
    if "date" in d0.columns:
        d0 = d0.sort_values("date")

    fe_terms = "C(month) + C(year) + C(dow)"

    # -------------------------
    # Helpers: stars + Stata-like formatting (SINGLE LINE)
    # -------------------------
    def stars(p):
        if p < 0.01:
            return "***"
        elif p < 0.05:
            return "**"
        elif p < 0.10:
            return "*"
        return ""

    def fmt_stata(res, var, dec=3):
        """
        SINGLE-LINE cell: 'coef*** (se)' so no '\n' appears.
        """
        if (not hasattr(res, "params")) or (not hasattr(res.params, "index")) or (var not in res.params.index):
            return ""
        coef = float(res.params[var])

        # some results may not have bse/pvalues for every var depending on cov_type
        try:
            se = float(res.bse[var])
        except Exception:
            se = np.nan
        try:
            p = float(res.pvalues[var])
        except Exception:
            p = np.nan

        st = "" if (not np.isfinite(p)) else stars(p)
        if (not np.isfinite(se)):
            return f"{coef:.{dec}f}{st} (.)"
        return f"{coef:.{dec}f}{st} ({se:.{dec}f})"

    def nobs(res):
        return int(res.nobs) if hasattr(res, "nobs") else np.nan

    # -------------------------
    # Column (1): reg occurrence_pal daily_woi ... vce(cluster monthyear)
    # -------------------------
    needed1 = ["occurrence_pal", "daily_woi", "month", "year", "dow", "monthyear"]
    s1 = d0[needed1].dropna().copy()
    m1 = smf.ols(f"occurrence_pal ~ daily_woi + {fe_terms}", data=s1).fit(
        cov_type="cluster", cov_kwds={"groups": s1["monthyear"]}
    )

    # -------------------------
    # Column (2): newey occurrence_pal daily_woi leaddaily_woi ... lag(7)
    # -------------------------
    needed2 = ["occurrence_pal", "daily_woi", "leaddaily_woi", "month", "year", "dow"]
    if "date" in d0.columns:
        needed2 += ["date"]
    s2 = d0[needed2].dropna().copy()
    if "date" in s2.columns:
        s2 = s2.sort_values("date")
    m2 = smf.ols(f"occurrence_pal ~ daily_woi + leaddaily_woi + {fe_terms}", data=s2).fit(
        cov_type="HAC", cov_kwds={"maxlags": 7}
    )

    # -------------------------
    # Column (3): newey occurrence_pal daily_woi leaddaily_woi lags + occurrence bins ... lag(7)
    # -------------------------
    rhs_lags = "lagdaily_woi + lagdaily_woi2 + lagdaily_woi3 + lagdaily_woi4 + lagdaily_woi5 + lagdaily_woi6 + lagdaily_woi7"
    rhs_occ  = "occurrence_1 + occurrence_2_7 + occurrence_8_14"

    needed3 = [
        "occurrence_pal", "daily_woi", "leaddaily_woi",
        "lagdaily_woi", "lagdaily_woi2", "lagdaily_woi3", "lagdaily_woi4", "lagdaily_woi5", "lagdaily_woi6", "lagdaily_woi7",
        "occurrence_1", "occurrence_2_7", "occurrence_8_14",
        "month", "year", "dow"
    ]
    if "date" in d0.columns:
        needed3 += ["date"]
    s3 = d0[needed3].dropna().copy()
    if "date" in s3.columns:
        s3 = s3.sort_values("date")
    m3 = smf.ols(
        f"occurrence_pal ~ daily_woi + leaddaily_woi + {rhs_lags} + {rhs_occ} + {fe_terms}",
        data=s3
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # -------------------------
    # Column (4): reg lnvic_pal daily_woi ... vce(cluster monthyear)
    # -------------------------
    needed4 = ["lnvic_pal", "daily_woi", "month", "year", "dow", "monthyear"]
    s4 = d0[needed4].dropna().copy()
    m4 = smf.ols(f"lnvic_pal ~ daily_woi + {fe_terms}", data=s4).fit(
        cov_type="cluster", cov_kwds={"groups": s4["monthyear"]}
    )

    # -------------------------
    # Column (5): newey lnvic_pal daily_woi leaddaily_woi ... lag(7)
    # -------------------------
    needed5 = ["lnvic_pal", "daily_woi", "leaddaily_woi", "month", "year", "dow"]
    if "date" in d0.columns:
        needed5 += ["date"]
    s5 = d0[needed5].dropna().copy()
    if "date" in s5.columns:
        s5 = s5.sort_values("date")
    m5 = smf.ols(f"lnvic_pal ~ daily_woi + leaddaily_woi + {fe_terms}", data=s5).fit(
        cov_type="HAC", cov_kwds={"maxlags": 7}
    )

    # -------------------------
    # Column (6): newey lnvic_pal daily_woi leaddaily_woi lags + occurrence bins ... lag(7)
    # -------------------------
    needed6 = [
        "lnvic_pal", "daily_woi", "leaddaily_woi",
        "lagdaily_woi", "lagdaily_woi2", "lagdaily_woi3", "lagdaily_woi4", "lagdaily_woi5", "lagdaily_woi6", "lagdaily_woi7",
        "occurrence_1", "occurrence_2_7", "occurrence_8_14",
        "month", "year", "dow"
    ]
    if "date" in d0.columns:
        needed6 += ["date"]
    s6 = d0[needed6].dropna().copy()
    if "date" in s6.columns:
        s6 = s6.sort_values("date")
    m6 = smf.ols(
        f"lnvic_pal ~ daily_woi + leaddaily_woi + {rhs_lags} + {rhs_occ} + {fe_terms}",
        data=s6
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # -------------------------
    # Column (7): NB GLM victims_pal with HAC(7)
    # -------------------------
    needed7 = [
        "victims_pal", "daily_woi", "leaddaily_woi",
        "lagdaily_woi", "lagdaily_woi2", "lagdaily_woi3", "lagdaily_woi4", "lagdaily_woi5", "lagdaily_woi6", "lagdaily_woi7",
        "occurrence_1", "occurrence_2_7", "occurrence_8_14",
        "month", "year", "dow"
    ]
    if "date" in d0.columns:
        needed7 += ["date"]
    s7 = d0[needed7].dropna().copy()
    if "date" in s7.columns:
        s7 = s7.sort_values("date")

    m7 = smf.glm(
        f"victims_pal ~ daily_woi + leaddaily_woi + {rhs_lags} + {rhs_occ} + {fe_terms}",
        data=s7,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    # -------------------------
    # Build outreg2-like table (keep vars)
    # -------------------------
    col_names = ["(1) Occurrence", "(2) Occurrence", "(3) Occurrence",
                 "(4) Ln(victims)", "(5) Ln(victims)", "(6) Ln(victims)",
                 "(7) Num. victims"]
    models = [m1, m2, m3, m4, m5, m6, m7]

    keep_vars = ["daily_woi", "leaddaily_woi", "lagdaily_woi",
                 "occurrence_1", "occurrence_2_7", "occurrence_8_14"]

    table = pd.DataFrame(index=keep_vars, columns=col_names)
    for cn, res in zip(col_names, models):
        for v in keep_vars:
            table.loc[v, cn] = fmt_stata(res, v, 3)

    stats = pd.DataFrame(index=["N"], columns=col_names)
    for cn, res in zip(col_names, models):
        stats.loc["N", cn] = nobs(res)

    # -------------------------
    # Export + display
    # -------------------------
    with pd.ExcelWriter(str(out_xlsx_path), engine="openpyxl") as writer:
        title_df = pd.DataFrame({"": [
            "*** MAIN TABLES ***",
            "Table 4. Palestinian Attacks and News Pressure",
            "",
            ""
        ]})
        title_df.to_excel(writer, sheet_name="Table 4", index=False, header=False, startrow=0)
        table.to_excel(writer, sheet_name="Table 4", startrow=6)
        stats.to_excel(writer, sheet_name="Table 4", startrow=6 + len(table) + 3)

    print(f"Saved: {out_xlsx_path}")

    # cleaner display
    display(table.astype(str).replace("nan", ""))
    display(stats.astype(str).replace("nan", ""))

    return table, stats

# RUN
table4, stats4 = run_table4(df, TABLES / "table_4.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_4.xlsx


,(1) Occurrence,(2) Occurrence,(3) Occurrence,(4) Ln(victims),(5) Ln(victims),(6) Ln(victims),(7) Num. victims
daily_woi,-0.008 (0.013),-0.015 (0.015),-0.004 (0.017),-0.014 (0.016),-0.023 (0.018),-0.016 (0.021),-0.427 (0.423)
leaddaily_woi,,0.013 (0.018),0.021 (0.018),,0.017 (0.019),0.023 (0.019),0.267 (0.329)
lagdaily_woi,,,-0.023 (0.021),,,-0.027 (0.024),-0.499 (0.471)
occurrence_1,,,0.013 (0.009),,,0.021** (0.010),0.352** (0.167)
occurrence_2_7,,,0.013** (0.006),,,0.012** (0.006),0.794* (0.407)
occurrence_8_14,,,0.006 (0.006),,,0.003 (0.006),0.293 (0.395)


,(1) Occurrence,(2) Occurrence,(3) Occurrence,(4) Ln(victims),(5) Ln(victims),(6) Ln(victims),(7) Num. victims
N,4048,4045,4024,4048,4045,4024,4024


$$ \textbf{Table 5 : Attacks and Next-day News Pressure Driven} \\ \textbf{by Predictable Political and Sports Events}$$

$$ \text{Table 5: panel A}$$

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS

# =========================
# Helpers: stars + formatting (outreg2 style) — SINGLE LINE
# =========================
def stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.10:
        return "*"
    return ""

def get_param(res, var):
    """
    Returns (coef, se, pval) for:
      - statsmodels OLS/GLM: res.params / res.bse / res.pvalues
      - linearmodels IV2SLS: res.params / res.std_errors / res.pvalues
    """
    if not hasattr(res, "params") or not hasattr(res.params, "index") or var not in res.params.index:
        return np.nan, np.nan, np.nan

    coef = float(res.params[var])

    if hasattr(res, "bse") and res.bse is not None and hasattr(res.bse, "index") and var in res.bse.index:
        se = float(res.bse[var])
    elif hasattr(res, "std_errors") and res.std_errors is not None and hasattr(res.std_errors, "index") and var in res.std_errors.index:
        se = float(res.std_errors[var])
    else:
        se = np.nan

    if hasattr(res, "pvalues") and res.pvalues is not None and hasattr(res.pvalues, "index") and var in res.pvalues.index:
        p = float(res.pvalues[var])
    else:
        # If pvalues missing, compute z-based p (ok for large samples)
        if np.isfinite(se) and se > 0:
            z = coef / se
            try:
                from scipy.stats import norm
                p = float(2.0 * (1.0 - norm.cdf(abs(z))))
            except Exception:
                p = float(2.0 * (1.0 - sm.stats.norm.cdf(abs(z))))
        else:
            p = np.nan

    return coef, se, p

def fmt_star(res, var, dec=3):
    """
    SINGLE-LINE cell (no '\n'):
      coef*** (se)
    """
    coef, se, p = get_param(res, var)
    if (not np.isfinite(coef)) or (not np.isfinite(se)):
        return ""
    st = "" if (not np.isfinite(p)) else stars(p)
    return f"{coef:.{dec}f}{st} ({se:.{dec}f})"

def cluster_F_single_restriction(res, restriction: str):
    """
    Practical cluster-robust F-like stat for a single restriction.
    """
    ft = res.f_test(restriction)
    if hasattr(ft, "fvalue") and ft.fvalue is not None:
        return float(np.asarray(ft.fvalue).squeeze())
    if hasattr(ft, "statistic") and ft.statistic is not None:
        return float(np.asarray(ft.statistic).squeeze())
    return np.nan

# =========================
# Panel A (Table 5A): Israel
# =========================
def run_table5_panelA(df_in: pd.DataFrame, out_xlsx_path):
    # Stata: sort date, if gaza_war==0
    d0 = df_in[df_in["gaza_war"] == 0].copy()
    if "date" in d0.columns:
        d0 = d0.sort_values("date")

    # FE: i.month i.year i.dow
    fe_terms = "C(month) + C(year) + C(dow)"
    pal_controls = "occurrence_pal_1 + occurrence_pal_2_7 + occurrence_pal_8_14"

    # -------------------------
    # (1) 1st stage corrected
    # -------------------------
    need1 = ["leaddaily_woi", "lead_maj_events",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s1 = d0[need1].dropna().copy()

    fs = smf.ols(
        f"leaddaily_woi ~ lead_maj_events + {pal_controls} + {fe_terms}",
        data=s1
    ).fit(cov_type="cluster", cov_kwds={"groups": s1["monthyear"]})

    F_cont = cluster_F_single_restriction(fs, "lead_maj_events = 0")

    # -------------------------
    # (2) 1st stage uncorrected
    # -------------------------
    need2 = ["leaddaily_woi_nc", "lead_maj_events",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s2 = d0[need2].dropna().copy()

    fs_nc = smf.ols(
        f"leaddaily_woi_nc ~ lead_maj_events + {pal_controls} + {fe_terms}",
        data=s2
    ).fit(cov_type="cluster", cov_kwds={"groups": s2["monthyear"]})

    F_cont_unc = cluster_F_single_restriction(fs_nc, "lead_maj_events = 0")

    # -------------------------
    # (3) IV 2SLS (occurrence, corrected)
    # -------------------------
    need3 = ["occurrence", "leaddaily_woi", "lead_maj_events",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s3 = d0[need3].dropna().copy()

    iv_exog = f"1 + {pal_controls} + {fe_terms}"
    iv = IV2SLS.from_formula(
        f"occurrence ~ {iv_exog} + [leaddaily_woi ~ lead_maj_events]",
        data=s3
    ).fit(cov_type="clustered", clusters=s3["monthyear"])

    # -------------------------
    # (4) IV 2SLS (occurrence, uncorrected)
    # -------------------------
    need4 = ["occurrence", "leaddaily_woi_nc", "lead_maj_events",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s4 = d0[need4].dropna().copy()

    iv_nc = IV2SLS.from_formula(
        f"occurrence ~ {iv_exog} + [leaddaily_woi_nc ~ lead_maj_events]",
        data=s4
    ).fit(cov_type="clustered", clusters=s4["monthyear"])

    # -------------------------
    # (5) Reduced form OLS (occurrence on lead_maj_events)
    # -------------------------
    need5 = ["occurrence", "lead_maj_events",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s5 = d0[need5].dropna().copy()

    rf = smf.ols(
        f"occurrence ~ lead_maj_events + {pal_controls} + {fe_terms}",
        data=s5
    ).fit(cov_type="cluster", cov_kwds={"groups": s5["monthyear"]})

    # -------------------------
    # (6) NB GLM victims_isr on leaddaily_woi (cluster monthyear)
    # -------------------------
    need6 = ["victims_isr", "leaddaily_woi",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s6 = d0[need6].dropna().copy()

    nb = smf.glm(
        f"victims_isr ~ leaddaily_woi + {pal_controls} + {fe_terms}",
        data=s6,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s6["monthyear"]})

    # -------------------------
    # (7) NB GLM victims_isr on leaddaily_woi_nc (cluster)
    # -------------------------
    need7 = ["victims_isr", "leaddaily_woi_nc",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s7 = d0[need7].dropna().copy()

    nb_nc = smf.glm(
        f"victims_isr ~ leaddaily_woi_nc + {pal_controls} + {fe_terms}",
        data=s7,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s7["monthyear"]})

    # -------------------------
    # (8) NB GLM reduced form victims_isr on lead_maj_events (cluster)
    # -------------------------
    need8 = ["victims_isr", "lead_maj_events",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "month", "year", "dow", "monthyear"]
    s8 = d0[need8].dropna().copy()

    nb_rf = smf.glm(
        f"victims_isr ~ lead_maj_events + {pal_controls} + {fe_terms}",
        data=s8,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s8["monthyear"]})

    # =========================
    # Build outreg2-like table (Panel A) with multiple rows
    # =========================
    cols = [
        "NP t+1, 1st stage",
        "NP t+1 (uncorrected), 1st stage",
        "Occurrence, IV 2nd stage",
        "Occurrence, IV 2nd stage (uncorr)",
        "Occurrence, Reduced form",
        "Num. victims, IV 2nd stage",
        "Num. victims, IV 2nd stage (uncorr)",
        "Num. victims, Reduced form",
    ]

    table = pd.DataFrame(index=[
        "Occurrence of major US political events or FIFA World Cup",
        "News pressure (t+1)",
        "News pressure uncorrected (t+1)",
    ], columns=cols)

    table.loc["Occurrence of major US political events or FIFA World Cup", cols[0]] = fmt_star(fs, "lead_maj_events")
    table.loc["Occurrence of major US political events or FIFA World Cup", cols[1]] = fmt_star(fs_nc, "lead_maj_events")
    table.loc["News pressure (t+1)", cols[2]] = fmt_star(iv, "leaddaily_woi")
    table.loc["News pressure uncorrected (t+1)", cols[3]] = fmt_star(iv_nc, "leaddaily_woi_nc")
    table.loc["Occurrence of major US political events or FIFA World Cup", cols[4]] = fmt_star(rf, "lead_maj_events")
    table.loc["News pressure (t+1)", cols[5]] = fmt_star(nb, "leaddaily_woi")
    table.loc["News pressure uncorrected (t+1)", cols[6]] = fmt_star(nb_nc, "leaddaily_woi_nc")
    table.loc["Occurrence of major US political events or FIFA World Cup", cols[7]] = fmt_star(nb_rf, "lead_maj_events")

    stats = pd.DataFrame(index=["N", "F excl. instr."], columns=cols)

    stats.loc["N", cols[0]] = int(fs.nobs)
    stats.loc["N", cols[1]] = int(fs_nc.nobs)
    stats.loc["N", cols[2]] = int(iv.nobs)
    stats.loc["N", cols[3]] = int(iv_nc.nobs)
    stats.loc["N", cols[4]] = int(rf.nobs)
    stats.loc["N", cols[5]] = int(nb.nobs)
    stats.loc["N", cols[6]] = int(nb_nc.nobs)
    stats.loc["N", cols[7]] = int(nb_rf.nobs)

    stats.loc["F excl. instr.", cols[0]] = F_cont
    stats.loc["F excl. instr.", cols[1]] = F_cont_unc
    stats.loc["F excl. instr.", cols[2]] = F_cont
    stats.loc["F excl. instr.", cols[3]] = F_cont_unc
    stats.loc["F excl. instr.", cols[5]] = F_cont
    stats.loc["F excl. instr.", cols[6]] = F_cont_unc

    # Export + display
    with pd.ExcelWriter(str(out_xlsx_path), engine="openpyxl") as writer:
        title_df = pd.DataFrame({"": [
            "*** MAIN TABLES ***",
            "Table 5A. Israeli Attacks and Predictable Newsworthy Events",
            "",
            ""
        ]})
        title_df.to_excel(writer, sheet_name="Table 5A", index=False, header=False, startrow=0)
        table.to_excel(writer, sheet_name="Table 5A", startrow=6)
        stats.to_excel(writer, sheet_name="Table 5A", startrow=6 + len(table) + 3)

    print(f"Saved: {out_xlsx_path}")

    # cleaner display
    display(table.astype(str).replace("nan", ""))
    display(stats.astype(str).replace("nan", ""))

    return table, stats

# ---- RUN (Panel A only) ----
table5a, stats5a = run_table5_panelA(df, TABLES / "table_5a.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_5a.xlsx


,"NP t+1, 1st stage","NP t+1 (uncorrected), 1st stage","Occurrence, IV 2nd stage","Occurrence, IV 2nd stage (uncorr)","Occurrence, Reduced form","Num. victims, IV 2nd stage","Num. victims, IV 2nd stage (uncorr)","Num. victims, Reduced form"
Occurrence of major US political events or FIFA World Cup,0.177*** (0.035),0.190*** (0.034),,,0.109*** (0.041),,,0.440*** (0.167)
News pressure (t+1),,,0.615** (0.244),,,0.482*** (0.133),,
News pressure uncorrected (t+1),,,,0.574*** (0.219),,,0.191 (0.145),


,"NP t+1, 1st stage","NP t+1 (uncorrected), 1st stage","Occurrence, IV 2nd stage","Occurrence, IV 2nd stage (uncorr)","Occurrence, Reduced form","Num. victims, IV 2nd stage","Num. victims, IV 2nd stage (uncorr)","Num. victims, Reduced form"
N,4046,4046,4046,4046,4049,4046,4046,4049
F excl. instr.,25.040856801998647,31.675092102731334,25.040856801998647,31.675092102731334,,25.040856801998647,31.675092102731334,


$$ \text{Table 5: panel B}$$

In [ ]:
# =========================
# Panel B (Table 5B): Palestinian Attacks
# - Same as your code, but NO "\n" in coefficient cells
# =========================

from IPython.display import display
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS

# -------------------------
# Helpers (shared)
# -------------------------
def stars_from_p(p):
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

def pval_from_z(z):
    try:
        from scipy.stats import norm
        return float(2.0 * (1.0 - norm.cdf(abs(z))))
    except Exception:
        return float(2.0 * (1.0 - sm.stats.norm.cdf(abs(z))))

def fmt_star(res, var, dec=3):
    """
    SINGLE-LINE formatting (no '\n'):
      coef*** (se)
    Works for statsmodels results and linearmodels IV2SLS results.
    """
    # coef
    if hasattr(res, "params") and hasattr(res.params, "index"):
        if var not in res.params.index:
            return ""
        b = float(res.params[var])
    else:
        names = list(getattr(res.model, "exog_names", []))
        if var not in names:
            return ""
        j = names.index(var)
        b = float(np.asarray(res.params)[j])

    # SE: statsmodels uses .bse, linearmodels uses .std_errors
    se = None
    if hasattr(res, "bse") and res.bse is not None:
        try:
            if hasattr(res.bse, "index") and var in res.bse.index:
                se = float(res.bse[var])
            else:
                j = list(res.params.index).index(var)
                se = float(np.asarray(res.bse)[j])
        except Exception:
            se = None

    if se is None and hasattr(res, "std_errors") and res.std_errors is not None:
        try:
            if hasattr(res.std_errors, "index") and var in res.std_errors.index:
                se = float(res.std_errors[var])
            else:
                j = list(res.params.index).index(var)
                se = float(np.asarray(res.std_errors)[j])
        except Exception:
            se = None

    if se is None or (not np.isfinite(se)) or se <= 0:
        return f"{b:.{dec}f} (.)"

    p = pval_from_z(b / se)
    st = stars_from_p(p)
    return f"{b:.{dec}f}{st} ({se:.{dec}f})"

def cluster_F_single_restriction(res, hypothesis):
    """
    Cluster-robust F-stat for a single restriction (statsmodels), e.g. "lead_maj_events = 0"
    """
    try:
        w = res.wald_test(hypothesis, use_f=True)
        stat = w.statistic
        if np.ndim(stat) > 0:
            stat = stat[0]
        return float(stat)
    except Exception:
        return np.nan

# =========================
# Panel B (Table 5B)
# =========================
def run_table5_panelB(df_in: pd.DataFrame, out_xlsx_path):
    # Stata: sort date, if gaza_war==0
    d0 = df_in[df_in["gaza_war"] == 0].copy()
    if "date" in d0.columns:
        d0 = d0.sort_values("date")

    # FE per Stata Panel B: i.year i.dow (NO i.month)
    fe_terms = "C(year) + C(dow)"
    isr_controls = "occurrence_1 + occurrence_2_7 + occurrence_8_14"

    # -------------------------
    # (1) 1st stage corrected
    # -------------------------
    need1 = ["leaddaily_woi", "lead_maj_events",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s1 = d0[need1].dropna().copy()

    fs = smf.ols(
        f"leaddaily_woi ~ lead_maj_events + {isr_controls} + {fe_terms}",
        data=s1
    ).fit(cov_type="cluster", cov_kwds={"groups": s1["monthyear"]})

    F_cont = cluster_F_single_restriction(fs, "lead_maj_events = 0")

    # -------------------------
    # (2) 1st stage uncorrected
    # -------------------------
    need2 = ["leaddaily_woi_nc", "lead_maj_events",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s2 = d0[need2].dropna().copy()

    fs_nc = smf.ols(
        f"leaddaily_woi_nc ~ lead_maj_events + {isr_controls} + {fe_terms}",
        data=s2
    ).fit(cov_type="cluster", cov_kwds={"groups": s2["monthyear"]})

    F_cont_unc = cluster_F_single_restriction(fs_nc, "lead_maj_events = 0")

    # -------------------------
    # (3) IV 2SLS (occurrence_pal, corrected)
    # -------------------------
    need3 = ["occurrence_pal", "leaddaily_woi", "lead_maj_events",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s3 = d0[need3].dropna().copy()

    iv_exog = f"1 + {isr_controls} + {fe_terms}"
    iv = IV2SLS.from_formula(
        f"occurrence_pal ~ {iv_exog} + [leaddaily_woi ~ lead_maj_events]",
        data=s3
    ).fit(cov_type="clustered", clusters=s3["monthyear"])

    # -------------------------
    # (4) IV 2SLS (occurrence_pal, uncorrected)
    # -------------------------
    need4 = ["occurrence_pal", "leaddaily_woi_nc", "lead_maj_events",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s4 = d0[need4].dropna().copy()

    iv_nc = IV2SLS.from_formula(
        f"occurrence_pal ~ {iv_exog} + [leaddaily_woi_nc ~ lead_maj_events]",
        data=s4
    ).fit(cov_type="clustered", clusters=s4["monthyear"])

    # -------------------------
    # (5) Reduced form OLS (occurrence_pal on lead_maj_events)
    # -------------------------
    need5 = ["occurrence_pal", "lead_maj_events",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s5 = d0[need5].dropna().copy()

    rf = smf.ols(
        f"occurrence_pal ~ lead_maj_events + {isr_controls} + {fe_terms}",
        data=s5
    ).fit(cov_type="cluster", cov_kwds={"groups": s5["monthyear"]})

    # -------------------------
    # (6) NB GLM victims_pal on leaddaily_woi (cluster monthyear)
    # -------------------------
    need6 = ["victims_pal", "leaddaily_woi",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s6 = d0[need6].dropna().copy()

    nb = smf.glm(
        f"victims_pal ~ leaddaily_woi + {isr_controls} + {fe_terms}",
        data=s6,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s6["monthyear"]})

    # -------------------------
    # (7) NB GLM victims_pal on leaddaily_woi_nc (cluster)
    # -------------------------
    need7 = ["victims_pal", "leaddaily_woi_nc",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s7 = d0[need7].dropna().copy()

    nb_nc = smf.glm(
        f"victims_pal ~ leaddaily_woi_nc + {isr_controls} + {fe_terms}",
        data=s7,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s7["monthyear"]})

    # -------------------------
    # (8) NB GLM reduced form victims_pal on lead_maj_events (cluster)
    # -------------------------
    need8 = ["victims_pal", "lead_maj_events",
             "occurrence_1", "occurrence_2_7", "occurrence_8_14",
             "year", "dow", "monthyear"]
    s8 = d0[need8].dropna().copy()

    nb_rf = smf.glm(
        f"victims_pal ~ lead_maj_events + {isr_controls} + {fe_terms}",
        data=s8,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s8["monthyear"]})

    # =========================
    # Build outreg2-like table (Panel B)
    # =========================
    cols = [
        "NP t+1, 1st stage",
        "NP t+1 (uncorrected), 1st stage",
        "Occurrence, IV 2nd stage",
        "Occurrence, IV 2nd stage (uncorr)",
        "Occurrence, Reduced form",
        "Num. victims, IV 2nd stage",
        "Num. victims, IV 2nd stage (uncorr)",
        "Num. victims, Reduced form",
    ]

    table = pd.DataFrame(index=[
        "Occurrence of major US political events or FIFA World Cup",
        "News pressure (t+1)",
        "News pressure uncorrected (t+1)",
    ], columns=cols)

    table.loc["Occurrence of major US political events or FIFA World Cup", cols[0]] = fmt_star(fs, "lead_maj_events")
    table.loc["Occurrence of major US political events or FIFA World Cup", cols[1]] = fmt_star(fs_nc, "lead_maj_events")
    table.loc["News pressure (t+1)", cols[2]] = fmt_star(iv, "leaddaily_woi")
    table.loc["News pressure uncorrected (t+1)", cols[3]] = fmt_star(iv_nc, "leaddaily_woi_nc")
    table.loc["Occurrence of major US political events or FIFA World Cup", cols[4]] = fmt_star(rf, "lead_maj_events")
    table.loc["News pressure (t+1)", cols[5]] = fmt_star(nb, "leaddaily_woi")
    table.loc["News pressure uncorrected (t+1)", cols[6]] = fmt_star(nb_nc, "leaddaily_woi_nc")
    table.loc["Occurrence of major US political events or FIFA World Cup", cols[7]] = fmt_star(nb_rf, "lead_maj_events")

    stats = pd.DataFrame(index=["N", "F excl. instr."], columns=cols)

    stats.loc["N", cols[0]] = int(fs.nobs)
    stats.loc["N", cols[1]] = int(fs_nc.nobs)
    stats.loc["N", cols[2]] = int(iv.nobs)
    stats.loc["N", cols[3]] = int(iv_nc.nobs)
    stats.loc["N", cols[4]] = int(rf.nobs)
    stats.loc["N", cols[5]] = int(nb.nobs)
    stats.loc["N", cols[6]] = int(nb_nc.nobs)
    stats.loc["N", cols[7]] = int(nb_rf.nobs)

    stats.loc["F excl. instr.", cols[0]] = F_cont
    stats.loc["F excl. instr.", cols[1]] = F_cont_unc
    stats.loc["F excl. instr.", cols[2]] = F_cont
    stats.loc["F excl. instr.", cols[3]] = F_cont_unc
    stats.loc["F excl. instr.", cols[5]] = F_cont
    stats.loc["F excl. instr.", cols[6]] = F_cont_unc

    with pd.ExcelWriter(str(out_xlsx_path), engine="openpyxl") as writer:
        title_df = pd.DataFrame({"": [
            "*** MAIN TABLES ***",
            "Table 5B. Palestinian Attacks and Predictable Newsworthy Events",
            "",
            ""
        ]})
        title_df.to_excel(writer, sheet_name="Table 5B", index=False, header=False, startrow=0)
        table.to_excel(writer, sheet_name="Table 5B", startrow=6)
        stats.to_excel(writer, sheet_name="Table 5B", startrow=6 + len(table) + 3)

    print(f"Saved: {out_xlsx_path}")

    # Cleaner display (avoid fillna dtype warning)
    display(table.astype(str).replace("nan", ""))
    display(stats.astype(str).replace("nan", ""))

    return table, stats

# ---- RUN (Panel B) ----
table5b, stats5b = run_table5_panelB(df, TABLES / "table_5b.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_5b.xlsx


,"NP t+1, 1st stage","NP t+1 (uncorrected), 1st stage","Occurrence, IV 2nd stage","Occurrence, IV 2nd stage (uncorr)","Occurrence, Reduced form","Num. victims, IV 2nd stage","Num. victims, IV 2nd stage (uncorr)","Num. victims, Reduced form"
Occurrence of major US political events or FIFA World Cup,0.139*** (0.040),0.152*** (0.040),,,-0.018 (0.014),,,0.114 (0.290)
News pressure (t+1),,,-0.132 (0.101),,,-0.026 (0.258),,
News pressure uncorrected (t+1),,,,-0.121 (0.089),,,-0.475 (0.298),


,"NP t+1, 1st stage","NP t+1 (uncorrected), 1st stage","Occurrence, IV 2nd stage","Occurrence, IV 2nd stage (uncorr)","Occurrence, Reduced form","Num. victims, IV 2nd stage","Num. victims, IV 2nd stage (uncorr)","Num. victims, Reduced form"
N,4046,4046,4046,4046,4049,4046,4046,4049
F excl. instr.,11.89987854067023,14.727098712230509,11.89987854067023,14.727098712230509,,11.89987854067023,14.727098712230509,


$$ \text{Table 5: panel C}$$

In [ ]:
# ============================================================
# Table 5C (Panel C) — Placebo Israel
# Goal: same as your code, but NO "\n" in coefficient cells
# ============================================================

from IPython.display import display
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS

# -------------------------
# Helpers
# -------------------------
def stars_from_p(p):
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

def pval_from_z(z):
    try:
        from scipy.stats import norm
        return float(2.0 * (1.0 - norm.cdf(abs(z))))
    except Exception:
        return float(2.0 * (1.0 - sm.stats.norm.cdf(abs(z))))

def fmt_star(res, var, dec=3):
    """
    SINGLE-LINE formatting:
    coef*** (se)
    (no '\n' anywhere)
    """
    # linearmodels / statsmodels both expose params & std_errors/bse differently
    # We'll try robustly.
    if hasattr(res, "params") and hasattr(res.params, "index"):
        if var not in res.params.index:
            return ""
        b = float(res.params[var])
    else:
        # ndarray-like
        names = list(getattr(res.model, "exog_names", []))
        if var not in names:
            return ""
        j = names.index(var)
        b = float(np.asarray(res.params)[j])

    # standard errors
    se = None
    if hasattr(res, "bse"):
        if hasattr(res.bse, "index") and var in res.bse.index:
            se = float(res.bse[var])
        else:
            try:
                j = list(res.params.index).index(var)
                se = float(np.asarray(res.bse)[j])
            except Exception:
                se = None

    if se is None and hasattr(res, "std_errors"):
        if hasattr(res.std_errors, "index") and var in res.std_errors.index:
            se = float(res.std_errors[var])
        else:
            try:
                j = list(res.params.index).index(var)
                se = float(np.asarray(res.std_errors)[j])
            except Exception:
                se = None

    if se is None or (not np.isfinite(se)) or se <= 0:
        return f"{b:.{dec}f} (.)"

    p = pval_from_z(b / se)
    st = stars_from_p(p)
    return f"{b:.{dec}f}{st} ({se:.{dec}f})"

def cluster_F_single_restriction(res, hypothesis):
    """
    Cluster-robust F-stat for a single restriction, e.g. "lead_disaster = 0"
    Uses statsmodels wald_test with use_f=True.
    Returns the F statistic as float (or np.nan if not available).
    """
    try:
        w = res.wald_test(hypothesis, use_f=True)
        # statsmodels returns an object with .statistic
        stat = w.statistic
        if np.ndim(stat) > 0:
            stat = stat[0]
        return float(stat)
    except Exception:
        return np.nan

# ============================================================
# Main function
# ============================================================
def run_table5_panelC_placebo_israel(df_in: pd.DataFrame, out_xlsx_path):
    # Stata: if gaza_war==0
    d0 = df_in[df_in["gaza_war"] == 0].copy()
    if "date" in d0.columns:
        d0 = d0.sort_values("date")

    fe_terms = "C(dow) + C(month) + C(year)"
    pal_controls = "occurrence_pal_1 + occurrence_pal_2_7 + occurrence_pal_8_14"

    # -------------------------
    # (1) 1st stage corrected: leaddaily_woi on lead_disaster
    # -------------------------
    need1 = ["leaddaily_woi", "lead_disaster", "lead_killed_1000",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s1 = d0[need1].dropna().copy()

    fs = smf.ols(
        f"leaddaily_woi ~ lead_disaster + {pal_controls} + {fe_terms}",
        data=s1
    ).fit(cov_type="cluster", cov_kwds={"groups": s1["monthyear"]})

    Fd_cont = cluster_F_single_restriction(fs, "lead_disaster = 0")

    # -------------------------
    # (2) 1st stage uncorrected: leaddaily_woi_nc on lead_disaster
    # -------------------------
    need2 = ["leaddaily_woi_nc", "lead_disaster", "lead_killed_1000",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s2 = d0[need2].dropna().copy()

    fs_nc = smf.ols(
        f"leaddaily_woi_nc ~ lead_disaster + {pal_controls} + {fe_terms}",
        data=s2
    ).fit(cov_type="cluster", cov_kwds={"groups": s2["monthyear"]})

    Fd_cont_nc = cluster_F_single_restriction(fs_nc, "lead_disaster = 0")

    # -------------------------
    # (3) IV 2SLS: occurrence on leaddaily_woi instrumented by lead_disaster
    # -------------------------
    need3 = ["occurrence", "leaddaily_woi", "lead_disaster",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s3 = d0[need3].dropna().copy()

    iv_exog = f"1 + {pal_controls} + {fe_terms}"
    iv = IV2SLS.from_formula(
        f"occurrence ~ {iv_exog} + [leaddaily_woi ~ lead_disaster]",
        data=s3
    ).fit(cov_type="clustered", clusters=s3["monthyear"])

    # -------------------------
    # (4) IV 2SLS: occurrence on leaddaily_woi_nc instrumented by lead_disaster
    # -------------------------
    need4 = ["occurrence", "leaddaily_woi_nc", "lead_disaster",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s4 = d0[need4].dropna().copy()

    iv_nc = IV2SLS.from_formula(
        f"occurrence ~ {iv_exog} + [leaddaily_woi_nc ~ lead_disaster]",
        data=s4
    ).fit(cov_type="clustered", clusters=s4["monthyear"])

    # -------------------------
    # (5) Reduced form OLS: occurrence on lead_disaster
    # -------------------------
    need5 = ["occurrence", "lead_disaster",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s5 = d0[need5].dropna().copy()

    rf = smf.ols(
        f"occurrence ~ lead_disaster + {pal_controls} + {fe_terms}",
        data=s5
    ).fit(cov_type="cluster", cov_kwds={"groups": s5["monthyear"]})

    # -------------------------
    # (6) NB GLM: victims_isr on leaddaily_woi
    # -------------------------
    need6 = ["victims_isr", "leaddaily_woi",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s6 = d0[need6].dropna().copy()

    nb = smf.glm(
        f"victims_isr ~ leaddaily_woi + {pal_controls} + {fe_terms}",
        data=s6,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s6["monthyear"]})

    # -------------------------
    # (7) NB GLM: victims_isr on leaddaily_woi_nc
    # -------------------------
    need7 = ["victims_isr", "leaddaily_woi_nc",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s7 = d0[need7].dropna().copy()

    nb_nc = smf.glm(
        f"victims_isr ~ leaddaily_woi_nc + {pal_controls} + {fe_terms}",
        data=s7,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s7["monthyear"]})

    # -------------------------
    # (8) NB GLM reduced form: victims_isr on lead_disaster
    # -------------------------
    need8 = ["victims_isr", "lead_disaster", "lead_killed_1000",
             "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14",
             "dow", "month", "year", "monthyear"]
    s8 = d0[need8].dropna().copy()

    nb_rf = smf.glm(
        f"victims_isr ~ lead_disaster + {pal_controls} + {fe_terms}",
        data=s8,
        family=sm.families.NegativeBinomial()
    ).fit(cov_type="cluster", cov_kwds={"groups": s8["monthyear"]})

    # =========================
    # Build table (Panel C)
    # =========================
    cols = [
        "NP-Tomorrow, 1st stage",
        "UncorNP-Tomorrow, 1st stage",
        "Occurrence, IV 2nd stage",
        "Occurrence, IV 2nd stage (uncorr)",
        "Occurrence, Reduced form",
        "Num. victims, IV 2nd stage",
        "Num. victims, IV 2nd stage (uncorr)",
        "Num. victims, Reduced form",
    ]

    table = pd.DataFrame(
        index=[
            "Unpredictable newsworthy events (placebo)",
            "News pressure (t+1)",
            "News pressure uncorrected (t+1)",
        ],
        columns=cols
    )

    # (1)-(2): coef on lead_disaster
    table.loc["Unpredictable newsworthy events (placebo)", cols[0]] = fmt_star(fs, "lead_disaster")
    table.loc["Unpredictable newsworthy events (placebo)", cols[1]] = fmt_star(fs_nc, "lead_disaster")

    # (3)-(4): coef on endogenous regressor
    table.loc["News pressure (t+1)", cols[2]] = fmt_star(iv, "leaddaily_woi")
    table.loc["News pressure uncorrected (t+1)", cols[3]] = fmt_star(iv_nc, "leaddaily_woi_nc")

    # (5): reduced form occurrence on lead_disaster
    table.loc["Unpredictable newsworthy events (placebo)", cols[4]] = fmt_star(rf, "lead_disaster")

    # (6)-(7): victims on leaddaily_woi / leaddaily_woi_nc
    table.loc["News pressure (t+1)", cols[5]] = fmt_star(nb, "leaddaily_woi")
    table.loc["News pressure uncorrected (t+1)", cols[6]] = fmt_star(nb_nc, "leaddaily_woi_nc")

    # (8): victims reduced form on lead_disaster
    table.loc["Unpredictable newsworthy events (placebo)", cols[7]] = fmt_star(nb_rf, "lead_disaster")

    stats = pd.DataFrame(index=["N", "F excl. instr. (cluster-robust)"], columns=cols)

    stats.loc["N", cols[0]] = int(fs.nobs)
    stats.loc["N", cols[1]] = int(fs_nc.nobs)
    stats.loc["N", cols[2]] = int(iv.nobs)
    stats.loc["N", cols[3]] = int(iv_nc.nobs)
    stats.loc["N", cols[4]] = int(rf.nobs)
    stats.loc["N", cols[5]] = int(nb.nobs)
    stats.loc["N", cols[6]] = int(nb_nc.nobs)
    stats.loc["N", cols[7]] = int(nb_rf.nobs)

    # F stats (use first-stage ones)
    stats.loc["F excl. instr. (cluster-robust)", cols[0]] = Fd_cont
    stats.loc["F excl. instr. (cluster-robust)", cols[1]] = Fd_cont_nc
    stats.loc["F excl. instr. (cluster-robust)", cols[2]] = Fd_cont
    stats.loc["F excl. instr. (cluster-robust)", cols[3]] = Fd_cont_nc
    stats.loc["F excl. instr. (cluster-robust)", cols[5]] = Fd_cont
    stats.loc["F excl. instr. (cluster-robust)", cols[6]] = Fd_cont_nc

    # Export + display
    with pd.ExcelWriter(str(out_xlsx_path), engine="openpyxl") as writer:
        title_df = pd.DataFrame({"": [
            "*** MAIN TABLES ***",
            "Table 5C (placebo). Israeli Attacks and Unpredictable Newsworthy Events",
            "",
            ""
        ]})
        title_df.to_excel(writer, sheet_name="Table 5C", index=False, header=False, startrow=0)
        table.to_excel(writer, sheet_name="Table 5C", startrow=6)
        stats.to_excel(writer, sheet_name="Table 5C", startrow=6 + len(table) + 3)

    print(f"Saved: {out_xlsx_path}")

    # avoid FutureWarning by not using fillna("") on mixed dtypes
    display(table.astype(str).replace("nan", ""))
    display(stats.astype(str).replace("nan", ""))

    return table, stats

# ---- RUN ----
run_table5_panelC_placebo_israel(df, TABLES / "table_5c.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_5c.xlsx


,"NP-Tomorrow, 1st stage","UncorNP-Tomorrow, 1st stage","Occurrence, IV 2nd stage","Occurrence, IV 2nd stage (uncorr)","Occurrence, Reduced form","Num. victims, IV 2nd stage","Num. victims, IV 2nd stage (uncorr)","Num. victims, Reduced form"
Unpredictable newsworthy events (placebo),0.082*** (0.025),0.078*** (0.026),,,-0.023 (0.045),,,-0.089 (0.162)
News pressure (t+1),,,-0.278 (0.567),,,0.482*** (0.133),,
News pressure uncorrected (t+1),,,,-0.291 (0.589),,,0.191 (0.145),


,"NP-Tomorrow, 1st stage","UncorNP-Tomorrow, 1st stage","Occurrence, IV 2nd stage","Occurrence, IV 2nd stage (uncorr)","Occurrence, Reduced form","Num. victims, IV 2nd stage","Num. victims, IV 2nd stage (uncorr)","Num. victims, Reduced form"
N,4046,4046,4046,4046,4049,4046,4046,4049
F excl. instr. (cluster-robust),10.709520677813291,9.111713520572835,10.709520677813291,9.111713520572835,,10.709520677813291,9.111713520572835,


(                                          NP-Tomorrow, 1st stage  \
 Unpredictable newsworthy events (placebo)       0.082*** (0.025)   
 News pressure (t+1)                                          NaN   
 News pressure uncorrected (t+1)                              NaN   
 
                                           UncorNP-Tomorrow, 1st stage  \
 Unpredictable newsworthy events (placebo)            0.078*** (0.026)   
 News pressure (t+1)                                               NaN   
 News pressure uncorrected (t+1)                                   NaN   
 
                                           Occurrence, IV 2nd stage  \
 Unpredictable newsworthy events (placebo)                      NaN   
 News pressure (t+1)                                 -0.278 (0.567)   
 News pressure uncorrected (t+1)                                NaN   
 
                                           Occurrence, IV 2nd stage (uncorr)  \
 Unpredictable newsworthy events (placebo)                

$$ \textbf{Table 6 : The Urgency of Attacks and the Likelihood of Civilian Casualties}$$

$$ \text{Table 6A}$$

In [ ]:
# ============================================================
# Table 6A (Stata -> Python)
# - 6A: mlogit ... baseoutcome(1) vce(cluster monthyear)
# Output: Excel + display table
# ============================================================

from IPython.display import display
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_cluster

# -------------------------
# Formatting helpers (outreg2-like) — SINGLE LINE (no '\n')
# -------------------------
def stars(p):
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

def pval_from_z(z):
    try:
        from scipy.stats import norm
        return float(2 * (1 - norm.cdf(abs(z))))
    except Exception:
        return float(2 * (1 - sm.stats.norm.cdf(abs(z))))

def chi2_sf(x, df=1):
    try:
        from scipy.stats import chi2
        return float(chi2.sf(x, df))
    except Exception:
        return np.nan

def fmt_cell(coef, se, p=None, dec=3):
    """
    Single-line cell: 'coef*** (se)' so no '\n' appears in Excel/display.
    """
    if coef is None or se is None or (not np.isfinite(coef)) or (not np.isfinite(se)):
        return ""
    st = "" if (p is None or (not np.isfinite(p))) else stars(p)
    return f"{coef:.{dec}f}{st} ({se:.{dec}f})"

def _require_cols(df, cols, context=""):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns {missing} {('in ' + context) if context else ''}")

# ============================================================
# 6A: MNLogit (mlogit) with baseoutcome(1) + cluster SE
# ============================================================

def _mnlogit_fit_cluster(df, dep, rhs_cols, fe_cols, cluster_col, base_outcome=1):
    need = [dep] + rhs_cols + fe_cols + [cluster_col]
    d = df[need].dropna().copy()

    # enforce numeric outcomes + baseoutcome(1)
    y_raw = pd.to_numeric(d[dep], errors="coerce")
    d = d.loc[y_raw.notna()].copy()
    y_raw = y_raw.loc[d.index].astype(int)

    cats = sorted(y_raw.unique().tolist())
    if base_outcome not in cats:
        raise ValueError(f"{dep}: base outcome {base_outcome} not found. Found {cats}")

    non_base = [c for c in cats if c != base_outcome]
    if len(non_base) == 0:
        raise ValueError(f"{dep}: no non-base outcomes. Found {cats}")

    ordered = [base_outcome] + sorted(non_base)
    mapping = {cat: i for i, cat in enumerate(ordered)}  # base -> 0
    y = y_raw.map(mapping).astype(int)

    # design matrix with FE as dummies (drop_first to avoid collinearity)
    X = pd.get_dummies(d[rhs_cols + fe_cols], columns=fe_cols, drop_first=True)
    X = X.apply(pd.to_numeric, errors="coerce").astype(float)
    X = sm.add_constant(X, has_constant="add")

    valid = np.isfinite(X).all(axis=1) & np.isfinite(y.values)
    X = X.loc[valid]
    y = y.loc[valid]
    groups = d.loc[X.index, cluster_col]

    # fit multinomial logit
    mod = sm.MNLogit(y, X)
    res = mod.fit(disp=False)

    # clustered covariance (monthyear)
    V = cov_cluster(res, groups)

    # params as ndarray
    P = np.asarray(res.params)
    K = X.shape[1]
    if P.shape[1] != K and P.shape[0] == K:
        P = P.T  # ensure (J-1, K)

    out_cats = ordered[1:]  # original labels excluding base
    Jm1 = len(out_cats)

    # reshape SE from vec(params) covariance (column-major blocks by outcome)
    se_vec = np.sqrt(np.diag(V))
    bse_matrix = se_vec.reshape((K, Jm1), order="F")

    # Wald test: [2]leaddaily_woi = [3]leaddaily_woi
    X_cols = list(X.columns)
    p_test = np.nan

    if (2 in out_cats) and (3 in out_cats) and ("leaddaily_woi" in X_cols):
        j2 = out_cats.index(2)
        j3 = out_cats.index(3)
        i_lead = X_cols.index("leaddaily_woi")

        beta_vec = P.T.reshape((-1,), order="F")  # (K*Jm1,)
        pos2 = j2 * K + i_lead
        pos3 = j3 * K + i_lead

        r = np.zeros(beta_vec.size)
        r[pos2] = 1.0
        r[pos3] = -1.0

        diff = float(r @ beta_vec)
        var_diff = float(r @ V @ r)

        if var_diff > 0 and np.isfinite(var_diff):
            wald = (diff * diff) / var_diff
            p_test = chi2_sf(wald, df=1)

    return {
        "res": res,
        "P": P,                 # (J-1, K)
        "bse": bse_matrix,      # (K, J-1)
        "X_cols": X_cols,
        "outcome_cols": out_cats,
        "cats_all": cats,
        "N": int(res.nobs),
        "p_test_2eq3": p_test,
    }

def run_table6a(df_in, out_path):
    d0 = df_in[df_in["gaza_war"] == 0].copy()

    lag_vars = ["lagdaily_woi"] + [f"lagdaily_woi{k}" for k in range(2, 8)]
    rhs_cols = ["leaddaily_woi", "occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14"] + lag_vars
    fe_cols = ["month", "year", "dow"]
    cluster_col = "monthyear"

    specs = [
        ("attacks_target", "Non-targeted"),
        ("attacks_fatal",  "Fatal"),
        ("attacks_hpd",    "HPD"),
        ("attacks_hw",     "HW"),
    ]

    # 3 columns per model: (1) base blank, (2) outcome 2, (3) outcome 3
    colnames = []
    for _, title in specs:
        colnames += [f"{title} (1)", f"{title} (2)", f"{title} (3)"]

    coef_row = pd.Series(index=colnames, dtype=object)
    n_row    = pd.Series(index=colnames, dtype=object)
    p_row    = pd.Series(index=colnames, dtype=object)

    for dep, title in specs:
        _require_cols(d0, [dep, cluster_col] + rhs_cols + fe_cols, context=f"Table6A {dep}")

        out = _mnlogit_fit_cluster(
            d0, dep=dep, rhs_cols=rhs_cols, fe_cols=fe_cols,
            cluster_col=cluster_col, base_outcome=1
        )

        P = out["P"]
        bse = out["bse"]
        X_cols = out["X_cols"]
        out_cats = out["outcome_cols"]
        N = out["N"]
        ptest = out["p_test_2eq3"]

        # base column blank
        coef_row[f"{title} (1)"] = ""

        if "leaddaily_woi" not in X_cols:
            coef_row[f"{title} (2)"] = ""
            coef_row[f"{title} (3)"] = ""
        else:
            i_lead = X_cols.index("leaddaily_woi")

            if 2 in out_cats:
                j2 = out_cats.index(2)
                b2 = float(P[j2, i_lead])
                se2 = float(bse[i_lead, j2])
                p2 = pval_from_z(b2 / se2) if se2 > 0 else np.nan
                coef_row[f"{title} (2)"] = fmt_cell(b2, se2, p2, dec=3)
            else:
                coef_row[f"{title} (2)"] = ""

            if 3 in out_cats:
                j3 = out_cats.index(3)
                b3 = float(P[j3, i_lead])
                se3 = float(bse[i_lead, j3])
                p3 = pval_from_z(b3 / se3) if se3 > 0 else np.nan
                coef_row[f"{title} (3)"] = fmt_cell(b3, se3, p3, dec=3)
            else:
                coef_row[f"{title} (3)"] = ""

        # footer rows
        for j in [1, 2, 3]:
            n_row[f"{title} ({j})"] = N
            p_row[f"{title} ({j})"] = "" if not np.isfinite(ptest) else round(ptest, 4)

    table = pd.DataFrame([coef_row], index=["News pressure (t+1)"])
    footer = pd.DataFrame([n_row, p_row], index=["Observations", "p value"])
    out_df = pd.concat([table, footer], axis=0)

    out_path = str(out_path)
    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        out_df.to_excel(writer, sheet_name="Table 6A")

    print("Saved:", out_path)
    display(out_df)
    return out_df

# ============================================================
# RUN TABLE 6A ONLY
# ============================================================
table6a = run_table6a(df, TABLES / "table_6_a.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_6_a.xlsx


,Non-targeted (1),Non-targeted (2),Non-targeted (3),Fatal (1),Fatal (2),Fatal (3),HPD (1),HPD (2),HPD (3),HW (1),HW (2),HW (3)
News pressure (t+1),,0.385 (0.431),0.457** (0.193),,0.066 (0.252),0.494 (0.301),,-0.022 (0.305),0.336 (0.243),,0.116 (0.247),0.630** (0.316)
Observations,4025.0,4025.0,4025.0,2485.0,2485.0,2485.0,2483.0,2483.0,2483.0,2449.0,2449.0,2449.0
p value,0.8701,0.8701,0.8701,0.1024,0.1024,0.1024,0.1509,0.1509,0.1509,0.0358,0.0358,0.0358


$$ \text{Table 6B}$$

In [ ]:
# ============================================================
# TABLE 6B (Stata -> Python)
# Stata:
#   xi: glm y leaddaily_woi lags controls i.month i.year i.dow
#       if gaza_war==0, family(nbinom ml) vce(hac nwest 7)
#
# Python:
#   statsmodels discrete NegativeBinomial (MLE alpha) + HAC NW(7)
#   with robust covariance fallback so SE isn't missing.
# ============================================================

# ---------- formatting ----------
def _stars(p):
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

def _norm_cdf(x):
    try:
        from scipy.stats import norm
        return norm.cdf(x)
    except Exception:
        return sm.stats.norm.cdf(x)

def _pval_from_z(z):
    return float(2.0 * (1.0 - _norm_cdf(abs(z))))

def _fmt_cell(coef, se, p=None, dec=3):
    """
    Returns a string with REAL newline, so it displays on 2 lines in notebook (with style)
    and in Excel (as line break inside the cell).
    """
    if coef is None or not np.isfinite(coef):
        return ""
    if se is None or not np.isfinite(se):
        # mimic Stata missing SE display
        return f"{coef:.{dec}f}\n(.)"
    st = "" if (p is None or not np.isfinite(p)) else _stars(p)
    return f"{coef:.{dec}f}{st}\n({se:.{dec}f})"

# ---------- utils ----------
def _require_cols(df, cols, context=""):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns {missing}" + (f" in {context}" if context else ""))

def _find_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _get_classic_cov(res):
    """
    Try hard to recover a usable 'classic' covariance matrix.
    This is the key to avoid (.) in the first column.
    """
    # 1) cov_params_default
    if hasattr(res, "cov_params_default") and res.cov_params_default is not None:
        try:
            V = np.asarray(res.cov_params_default, dtype=float)
            if V.ndim == 2:
                return V
        except Exception:
            pass

    # 2) normalized_cov_params * scale
    if hasattr(res, "normalized_cov_params") and res.normalized_cov_params is not None:
        try:
            Vn = np.asarray(res.normalized_cov_params, dtype=float)
            scale = float(getattr(res, "scale", 1.0))
            V = Vn * scale
            if V.ndim == 2:
                return V
        except Exception:
            pass

    # 3) Hessian inversion fallback (works often for discrete NB)
    try:
        H = res.model.hessian(res.params)
        H = np.asarray(H, dtype=float)
        V = np.linalg.pinv(-H)   # safer than inv
        if V.ndim == 2:
            return V
    except Exception:
        pass

    return None

def _safe_param_extract(res, name):
    """
    Return (coef, se, pval) for a named param if possible.
    Works whether params is Series or ndarray.
    """
    # params names
    if hasattr(res.params, "index"):
        idx = list(res.params.index)
        if name not in idx:
            return None, None, None
        j = idx.index(name)
        b = float(res.params[name])
    else:
        idx = list(res.model.exog_names)
        if name not in idx:
            return None, None, None
        j = idx.index(name)
        b = float(np.asarray(res.params)[j])

    # se
    se = None
    if hasattr(res, "bse") and res.bse is not None:
        try:
            if hasattr(res.bse, "index"):
                se = float(res.bse[name])
            else:
                se = float(np.asarray(res.bse)[j])
        except Exception:
            se = None

    # pval (z-based, consistent with robust)
    p = None
    if se is not None and np.isfinite(se) and se > 0:
        p = _pval_from_z(b / se)

    return b, se, p

def _nb_mle_hac(formula, data, nlags=7):
    """
    Fit NegativeBinomial MLE and attach HAC NW(nlags) covariance.
    If HAC covariance fails or yields NaN SE for a parameter, fallback to classic covariance.
    """
    model = smf.negativebinomial(formula, data=data)

    # fit MLE
    try:
        res = model.fit(disp=0, method="bfgs", maxiter=800)
    except Exception:
        res = model.fit(disp=0)

    # classic cov fallback
    V_classic = _get_classic_cov(res)

    # HAC cov (may fail in some builds / edge cases)
    V_hac = None
    try:
        V_hac = cov_hac_simple(res, nlags=nlags)
        V_hac = np.asarray(V_hac, dtype=float)
        if V_hac.ndim != 2:
            V_hac = None
    except Exception:
        V_hac = None

    # choose cov to use
    V_use = V_hac if V_hac is not None else V_classic

    # attach covariance and bse
    if V_use is not None:
        res.cov_params_default = V_use
        bse = np.sqrt(np.diag(V_use))
        if hasattr(res.params, "index"):
            res.bse = pd.Series(bse, index=res.params.index)
        else:
            res.bse = bse
    else:
        # if absolutely nothing works, keep whatever statsmodels had
        pass

    # If some SE are NaN/inf with HAC, patch those using classic
    if V_hac is not None and V_classic is not None:
        try:
            bse_hac = np.sqrt(np.diag(V_hac))
            bse_cl  = np.sqrt(np.diag(V_classic))
            if hasattr(res.params, "index"):
                bse_hac = pd.Series(bse_hac, index=res.params.index)
                bse_cl  = pd.Series(bse_cl,  index=res.params.index)
                bad = ~np.isfinite(bse_hac)
                if bad.any():
                    bse_hac.loc[bad] = bse_cl.loc[bad]
                res.bse = bse_hac
                res.cov_params_default = None  # we only care about res.bse now
            else:
                bad = ~np.isfinite(bse_hac)
                if bad.any():
                    bse_hac[bad] = bse_cl[bad]
                res.bse = bse_hac
        except Exception:
            pass

    return res

def run_table6b(df_in: pd.DataFrame, out_path):
    # filter sample
    d0 = df_in[df_in["gaza_war"] == 0].copy()

    # RHS terms (same as Stata)
    lag_vars = ["lagdaily_woi"] + [f"lagdaily_woi{k}" for k in range(2, 8)]
    controls = ["occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14"] + lag_vars
    fe_terms = ["C(month)", "C(year)", "C(dow)"]

    rhs = "leaddaily_woi + " + " + ".join(lag_vars) + " + " + " + ".join(
        ["occurrence_pal_1", "occurrence_pal_2_7", "occurrence_pal_8_14"] + fe_terms
    )

    # filters used in Stata for 2 rows
    fatal_flag = _find_col(d0, ["occurrence_fatal"])
    hw_flag    = _find_col(d0, ["occurrence_hw"])

    specs = [
        ("victims_target",      "Victims of targeted killings",       None),
        ("victims_non_target",  "Victims of non-targeted attacks",    None),
        ("non_fatal_victims",   "Injuries",                           (fatal_flag, 0)),
        ("fatal_victims",       "Fatalities",                         None),
        ("victims_lpd",         "Casualties in NDP areas",            None),
        ("victims_hpd",         "Casualties in DP areas",             None),
        ("victims_nhw",         "Casualties with light weapons",      (hw_flag, 0)),
        ("victims_hw",          "Casualties with heavy weapons",      None),
    ]

    cols = [title for _, title, _ in specs]
    coef_row = pd.Series(index=cols, dtype=object)
    n_row    = pd.Series(index=cols, dtype=object)

    for y, title, cond in specs:
        needed_cols = [y, "leaddaily_woi", "month", "year", "dow"] + controls
        _require_cols(d0, needed_cols, context=f"Table6B {y}")

        di = d0.copy()

        # apply Stata conditions
        if cond is not None:
            c, v = cond
            if c is None:
                raise KeyError(
                    f"Stata uses a filter for '{y}' (occurrence_fatal==0 or occurrence_hw==0), "
                    f"but I can't find the needed flag column in your df."
                )
            di = di[di[c] == v].copy()

        di = di[needed_cols].dropna().copy()
        n_row[title] = int(di.shape[0])

        if di.shape[0] == 0:
            coef_row[title] = ""
            continue

        formula = f"{y} ~ {rhs}"

        res = _nb_mle_hac(formula, di, nlags=7)

        b, se, p = _safe_param_extract(res, "leaddaily_woi")
        coef_row[title] = _fmt_cell(b, se, p, dec=3)

    out_df = pd.DataFrame([coef_row, n_row], index=["News pressure (t+1)", "Observations"])
    out_path = str(out_path)

    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        out_df.to_excel(writer, sheet_name="Table 6B")

    print("Saved:", out_path)

    # display nicely in notebook (real line breaks)
    display(out_df.style.set_properties(**{"white-space": "pre-line"}))

    return out_df

# ---- RUN ----
table6b = run_table6b(df, TABLES / "table_6_b.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_6_b.xlsx


,Victims of targeted killings,Victims of non-targeted attacks,Injuries,Fatalities,Casualties in NDP areas,Casualties in DP areas,Casualties with light weapons,Casualties with heavy weapons
News pressure (t+1),0.390 (.),0.515*** (0.159),-0.047 (0.179),0.712*** (0.243),-0.004 (0.155),0.605** (0.245),-0.137 (0.170),0.789** (0.359)
Observations,4025,4025,1825,2485,2483,2483,1962,2449


$$ \textbf{Table 7 : Google Search Volume, Conflict-related News, and Timing of Attacks}$$

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display

# =========================
# TABLE 7 — Python analogue of Stata:
# xi: newey ... monthyear i.dow i.month, lag(7) force
# - OLS + Newey-West HAC(7)
# - Adds stars like Stata
# - Adds rows: DOW FEs / Linear time trend
# =========================

def run_table7(df_in: pd.DataFrame, out_path):
    d = df_in.copy()

    # --- must sort by time like Stata: sort date ---
    if "date" not in d.columns:
        raise ValueError("Table 7 needs a 'date' column (used for time-order and lags).")
    d = d.sort_values("date").copy()

    # ----------------------------
    # (1) Create interactions (Stata)
    # ----------------------------
    d["conflict_news_isr_occ_t"]      = d["any_conflict_news"] * d["occurrence"]
    d["conflict_news_isr_occ_t_1"]    = d["any_conflict_news"] * d["occurrence"].shift(1)
    d["conflict_news_no_isr_occ_t_y"] = d["any_conflict_news"] * ((1 - d["occurrence"]) * (1 - d["occurrence"].shift(1)))

    d["l_conflict_news_isr_occ_t"]      = d["length_conflict_news"] * d["occurrence"]
    d["l_conflict_news_isr_occ_t_1"]    = d["length_conflict_news"] * d["occurrence"].shift(1)
    d["l_conflict_news_no_isr_occ_t_y"] = d["length_conflict_news"] * ((1 - d["occurrence"]) * (1 - d["occurrence"].shift(1)))

    # ----------------------------
    # (2) Lags of interactions (Stata: gen L`var' = l.`var')
    # ----------------------------
    for v in [
        "conflict_news_isr_occ_t", "conflict_news_isr_occ_t_1", "conflict_news_no_isr_occ_t_y",
        "l_conflict_news_isr_occ_t", "l_conflict_news_isr_occ_t_1", "l_conflict_news_no_isr_occ_t_y"
    ]:
        d["L" + v] = d[v].shift(1)

    # ----------------------------
    # (3) Lags for other regressors used in Table 7
    # ----------------------------
    def lag(df, var, k):
        return df[var].shift(k)

    for var in ["lnvic", "lnvic_pal", "occurrence", "occurrence_pal"]:
        d[f"L1_{var}"] = lag(d, var, 1)
        d[f"L2_{var}"] = lag(d, var, 2)

    # ----------------------------
    # Helpers: fit Newey-West (HAC) and do Wald tests
    # ----------------------------
    def fit_newey(formula, data):
        return smf.ols(formula, data=data).fit(cov_type="HAC", cov_kwds={"maxlags": 7})

    def wald_pval(res, hypothesis_str):
        return float(res.wald_test(hypothesis_str).pvalue)

    def stars(p):
        if p < 0.01: return "***"
        if p < 0.05: return "**"
        if p < 0.10: return "*"
        return ""

    def pval_from_z(z):
        try:
            from scipy.stats import norm
            return float(2.0 * (1.0 - norm.cdf(abs(z))))
        except Exception:
            return float(2.0 * (1.0 - sm.stats.norm.cdf(abs(z))))

    # FE controls in Stata table 7:
    if "dow" not in d.columns or "month" not in d.columns or "monthyear" not in d.columns:
        raise ValueError("Table 7 needs columns: 'dow', 'month', 'monthyear' (as in Stata).")

    fe_terms = "C(dow) + C(month) + monthyear"

    # ----------------------------
    # (4) Four regressions like Stata Table 7
    # ----------------------------
    f1 = (
        "conflict_searches ~ "
        "conflict_news_isr_occ_t + conflict_news_isr_occ_t_1 + conflict_news_no_isr_occ_t_y + "
        "lnvic + L1_lnvic + lnvic_pal + L1_lnvic_pal + "
        "occurrence + L1_occurrence + occurrence_pal + L1_occurrence_pal + "
        f"{fe_terms}"
    )

    f2 = (
        "conflict_searches ~ "
        "conflict_news_isr_occ_t + conflict_news_isr_occ_t_1 + conflict_news_no_isr_occ_t_y + "
        "Lconflict_news_isr_occ_t + Lconflict_news_isr_occ_t_1 + Lconflict_news_no_isr_occ_t_y + "
        "lnvic + L1_lnvic + L2_lnvic + "
        "lnvic_pal + L1_lnvic_pal + L2_lnvic_pal + "
        "occurrence + L1_occurrence + L2_occurrence + "
        "occurrence_pal + L1_occurrence_pal + L2_occurrence_pal + "
        f"{fe_terms}"
    )

    f3 = (
        "conflict_searches ~ "
        "l_conflict_news_isr_occ_t + l_conflict_news_isr_occ_t_1 + l_conflict_news_no_isr_occ_t_y + "
        "lnvic + L1_lnvic + lnvic_pal + L1_lnvic_pal + "
        "occurrence + L1_occurrence + occurrence_pal + L1_occurrence_pal + "
        f"{fe_terms}"
    )

    f4 = (
        "conflict_searches ~ "
        "l_conflict_news_isr_occ_t + l_conflict_news_isr_occ_t_1 + l_conflict_news_no_isr_occ_t_y + "
        "Ll_conflict_news_isr_occ_t + Ll_conflict_news_isr_occ_t_1 + Ll_conflict_news_no_isr_occ_t_y + "
        "lnvic + L1_lnvic + L2_lnvic + "
        "lnvic_pal + L1_lnvic_pal + L2_lnvic_pal + "
        "occurrence + L1_occurrence + L2_occurrence + "
        "occurrence_pal + L1_occurrence_pal + L2_occurrence_pal + "
        f"{fe_terms}"
    )

    r1 = fit_newey(f1, d)
    p1 = wald_pval(r1, "conflict_news_isr_occ_t = conflict_news_isr_occ_t_1")

    r2 = fit_newey(f2, d)
    p2_main = wald_pval(r2, "conflict_news_isr_occ_t = conflict_news_isr_occ_t_1")
    p2_lag  = wald_pval(r2, "Lconflict_news_isr_occ_t = Lconflict_news_isr_occ_t_1")
    p2_sum  = wald_pval(
        r2,
        "Lconflict_news_isr_occ_t + conflict_news_isr_occ_t = "
        "Lconflict_news_isr_occ_t_1 + conflict_news_isr_occ_t_1"
    )

    r3 = fit_newey(f3, d)
    p3 = wald_pval(r3, "l_conflict_news_isr_occ_t = l_conflict_news_isr_occ_t_1")

    r4 = fit_newey(f4, d)
    p4_main = wald_pval(r4, "l_conflict_news_isr_occ_t = l_conflict_news_isr_occ_t_1")
    p4_lag  = wald_pval(r4, "Ll_conflict_news_isr_occ_t = Ll_conflict_news_isr_occ_t_1")
    p4_sum  = wald_pval(
        r4,
        "Ll_conflict_news_isr_occ_t + l_conflict_news_isr_occ_t = "
        "Ll_conflict_news_isr_occ_t_1 + l_conflict_news_isr_occ_t_1"
    )

    # ----------------------------
    # (5) Build output table (coef + se + stars)
    # ----------------------------
    def coef_se_str(res, var, dec=3):
        if var not in res.params.index:
            return ""
        b = float(res.params[var])
        se = float(res.bse[var])
        p = pval_from_z(b / se) if (se is not None and np.isfinite(se) and se > 0) else np.nan
        st = "" if not np.isfinite(p) else stars(p)
        return f"{b:.{dec}f}{st}\n({se:.{dec}f})"

    cols = {
        "(1) Isr–Pal Conflict": {
            "conflict_news_isr_occ_t":   coef_se_str(r1, "conflict_news_isr_occ_t"),
            "conflict_news_isr_occ_t_1": coef_se_str(r1, "conflict_news_isr_occ_t_1"),
            "p_value (t = t-1)": p1,
            "N": int(r1.nobs),
            "R2": float(r1.rsquared),
        },
        "(2) Isr–Pal Conflict": {
            "conflict_news_isr_occ_t":    coef_se_str(r2, "conflict_news_isr_occ_t"),
            "conflict_news_isr_occ_t_1":  coef_se_str(r2, "conflict_news_isr_occ_t_1"),
            "Lconflict_news_isr_occ_t":   coef_se_str(r2, "Lconflict_news_isr_occ_t"),
            "Lconflict_news_isr_occ_t_1": coef_se_str(r2, "Lconflict_news_isr_occ_t_1"),
            "p_value (t = t-1)": p2_main,
            "p_value lag": p2_lag,
            "p_value sum": p2_sum,
            "N": int(r2.nobs),
            "R2": float(r2.rsquared),
        },
        "(3) Isr–Pal Conflict": {
            "l_conflict_news_isr_occ_t":   coef_se_str(r3, "l_conflict_news_isr_occ_t"),
            "l_conflict_news_isr_occ_t_1": coef_se_str(r3, "l_conflict_news_isr_occ_t_1"),
            "p_value (t = t-1)": p3,
            "N": int(r3.nobs),
            "R2": float(r3.rsquared),
        },
        "(4) Isr–Pal Conflict": {
            "l_conflict_news_isr_occ_t":    coef_se_str(r4, "l_conflict_news_isr_occ_t"),
            "l_conflict_news_isr_occ_t_1":  coef_se_str(r4, "l_conflict_news_isr_occ_t_1"),
            "Ll_conflict_news_isr_occ_t":   coef_se_str(r4, "Ll_conflict_news_isr_occ_t"),
            "Ll_conflict_news_isr_occ_t_1": coef_se_str(r4, "Ll_conflict_news_isr_occ_t_1"),
            "p_value (t = t-1)": p4_main,
            "p_value lag": p4_lag,
            "p_value sum": p4_sum,
            "N": int(r4.nobs),
            "R2": float(r4.rsquared),
        }
    }

    # order rows like the table
    row_order = [
        "conflict_news_isr_occ_t",
        "conflict_news_isr_occ_t_1",
        "Lconflict_news_isr_occ_t",
        "Lconflict_news_isr_occ_t_1",
        "l_conflict_news_isr_occ_t",
        "l_conflict_news_isr_occ_t_1",
        "Ll_conflict_news_isr_occ_t",
        "Ll_conflict_news_isr_occ_t_1",
    ]

    coef_table = pd.DataFrame({k: {r: v.get(r, "") for r in row_order} for k, v in cols.items()})

    stats_rows = ["N", "DOW FEs", "Linear time trend", "p_value (t = t-1)", "p_value lag", "p_value sum", "R2"]
    stats_table = pd.DataFrame({k: {r: v.get(r, "") for r in stats_rows} for k, v in cols.items()})

    # add the two requested lines (like Stata)
    stats_table.loc["DOW FEs"] = ["Yes", "Yes", "Yes", "Yes"]
    stats_table.loc["Linear time trend"] = ["Yes", "Yes", "Yes", "Yes"]

    # reorder stats rows after inserting
    stats_table = stats_table.loc[["N", "DOW FEs", "Linear time trend", "p_value (t = t-1)", "p_value lag", "p_value sum", "R2"]]

    # combine into one output like your other tables
    out_df = pd.concat([coef_table, stats_table], axis=0)

    # export
    out_path = str(out_path)
    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        out_df.to_excel(writer, sheet_name="Table 7")

    print("Saved:", out_path)

    # display nicely (line breaks inside cells)
    display(out_df.style.set_properties(**{"white-space": "pre-line"}))

    return out_df

# ---- RUN ----
table7 = run_table7(df, TABLES / "table_7.xlsx")


Saved: /content/drive/MyDrive/replication/tables/table_7.xlsx


,(1) Isr–Pal Conflict,(2) Isr–Pal Conflict,(3) Isr–Pal Conflict,(4) Isr–Pal Conflict
conflict_news_isr_occ_t,0.014 (0.113),-0.058 (0.125),,
conflict_news_isr_occ_t_1,0.368*** (0.099),0.295** (0.125),,
Lconflict_news_isr_occ_t,,0.047 (0.141),,
Lconflict_news_isr_occ_t_1,,0.329*** (0.118),,
l_conflict_news_isr_occ_t,,,-0.003 (0.034),-0.044 (0.039)
l_conflict_news_isr_occ_t_1,,,0.109*** (0.030),0.108*** (0.039)
Ll_conflict_news_isr_occ_t,,,,-0.034 (0.043)
Ll_conflict_news_isr_occ_t_1,,,,0.114*** (0.038)
N,2773,2741,2773,2741
DOW FEs,Yes,Yes,Yes,Yes


In [ ]:
tables_dir = Path("/content/drive/MyDrive/replication/tables")

for f in tables_dir.glob("*.xls*"):
    print(f.name)


table_1a.xlsx
table_2.xlsx
table_1b.xlsx
table_4.xlsx
table_3.xlsx
table_5b.xlsx
table_5a.xlsx
table_6_a.xlsx
table_6_b.xlsx
table_7.xlsx


$$ \textbf{Figure 1: Israeli and Palestinian Fatalities by Month}$$

In [ ]:
# ============================================================
# Figure 1 — Israeli and Palestinian Fatalities by Month
# Python translation of the Stata code
# Fixes included:
#   - No NameError (paths defined)
#   - No FileNotFoundError (auto-locate replication_file1.dta)
#   - Exports figure_1_a.pdf and figure_1_b.pdf
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# -------------------------
# (0) PATHS
# -------------------------
ROOT = Path(".").resolve()

DTA = ROOT / "dta"
FIGURES = ROOT / "figures"
FIGURES.mkdir(parents=True, exist_ok=True)

DTA_FILENAME = "replication_file1.dta"

def load_replication_file(root: Path, dta_dir: Path, filename: str) -> pd.DataFrame:
    """
    Tries these locations (in this order):
      1) ./dta/replication_file1.dta
      2) ./replication_file1.dta
      3) any subfolder under ROOT (recursive search)
    """
    candidates = [
        dta_dir / filename,
        root / filename,
    ]

    for p in candidates:
        if p.exists():
            print("Loaded data from:", p)
            return pd.read_stata(p)

    # recursive fallback
    hits = list(root.rglob(filename))
    if len(hits) == 1:
        print("Loaded data from:", hits[0])
        return pd.read_stata(hits[0])
    elif len(hits) > 1:
        print("Found multiple matches. Using the first one:")
        for h in hits:
            print(" -", h)
        print("Loaded data from:", hits[0])
        return pd.read_stata(hits[0])

    # if nothing found -> raise with helpful message
    raise FileNotFoundError(
        f"Could not find {filename}.\n"
        f"Tried:\n"
        f" - {dta_dir / filename}\n"
        f" - {root / filename}\n"
        f"And searched recursively under:\n"
        f" - {root}\n\n"
        f"Fix: either move the file into ./dta/ or set ROOT to the folder that contains it."
    )

# -------------------------
# (1) PREP MONTHLY DATA (collapse)
# -------------------------
def _prep_monthly(df: pd.DataFrame) -> pd.DataFrame:
    need = ["monthyear", "post_intifada", "victims_isr", "victims_pal"]
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns for Figure 1: {missing}")

    d = df[need].copy()

    monthly = (
        d.groupby(["monthyear", "post_intifada"], as_index=False)
         .agg(sum_vic=("victims_isr", "sum"),
              sum_vic_pal=("victims_pal", "sum"))
         .sort_values("monthyear")
         .reset_index(drop=True)
    )
    return monthly


def _add_regime_means(monthly: pd.DataFrame) -> pd.DataFrame:
    out = monthly.copy()

    m_pre_isr  = out.loc[out["post_intifada"] == 0, "sum_vic"].mean(skipna=True)
    m_post_isr = out.loc[out["post_intifada"] == 1, "sum_vic"].mean(skipna=True)
    m_pre_pal  = out.loc[out["post_intifada"] == 0, "sum_vic_pal"].mean(skipna=True)
    m_post_pal = out.loc[out["post_intifada"] == 1, "sum_vic_pal"].mean(skipna=True)

    out["mean_isr_pre"]  = np.where(out["post_intifada"] == 0, m_pre_isr,  np.nan)
    out["mean_isr_post"] = np.where(out["post_intifada"] == 1, m_post_isr, np.nan)
    out["mean_pal_pre"]  = np.where(out["post_intifada"] == 0, m_pre_pal,  np.nan)
    out["mean_pal_post"] = np.where(out["post_intifada"] == 1, m_post_pal, np.nan)

    return out


# -------------------------
# (2) PLOTTING
# -------------------------
def _plot_panel(
    monthly: pd.DataFrame,
    out_pdf_path: str,
    *,
    shade_blocks=None,
    intifada_end_monthyear=541,
    xlabel_ticks=None,
    xlabel_ticklabels=None,
):
    x = monthly["monthyear"].to_numpy()
    y_isr = monthly["sum_vic"].to_numpy()
    y_pal = monthly["sum_vic_pal"].to_numpy()

    fig, ax = plt.subplots(figsize=(10.5, 5.5))

    # Shaded blocks
    shade_handles, shade_labels = [], []
    if shade_blocks:
        for sb in shade_blocks:
            h = ax.axvspan(sb["x0"], sb["x1"], alpha=sb.get("alpha", 0.25))
            if sb.get("label"):
                shade_handles.append(h)
                shade_labels.append(sb["label"])

    # Main lines
    h1, = ax.plot(x, y_isr, marker="o", markersize=3, linewidth=1.2)
    h2, = ax.plot(x, y_pal, marker="o", markersize=2.5, linewidth=1.2, linestyle="--")

    # Mean lines
    ax.plot(x, monthly["mean_isr_pre"],  linewidth=1.0)
    ax.plot(x, monthly["mean_isr_post"], linewidth=1.0)
    ax.plot(x, monthly["mean_pal_pre"],  linewidth=1.0, linestyle="--")
    ax.plot(x, monthly["mean_pal_post"], linewidth=1.0, linestyle="--")

    # Intifada end line
    ax.axvline(intifada_end_monthyear, linewidth=1.0)

    ax.set_ylabel("Victims of Israeli and Palestinian attacks")
    ax.set_xlabel("")

    if xlabel_ticks is not None:
        ax.set_xticks(xlabel_ticks)
    if xlabel_ticklabels is not None:
        ax.set_xticklabels(xlabel_ticklabels)

    # Legend: shaded first, then series
    handles = shade_handles + [h1, h2]
    labels  = shade_labels + ["Victims of Israeli attacks", "Victims of Palestinian attacks"]
    ax.legend(handles, labels, frameon=False)

    fig.tight_layout()
    fig.savefig(out_pdf_path)
    plt.close(fig)


# -------------------------
# (3) MAIN — export Figure 1A / 1B
# -------------------------
def make_figure_1(df: pd.DataFrame, figures_dir: Path):
    monthly = _prep_monthly(df)

    # Panel A: Entire sample period
    panelA = monthly.copy()

    # End of second Intifada missing
    panelA.loc[panelA["monthyear"] == 541, ["sum_vic", "sum_vic_pal"]] = np.nan
    panelA = _add_regime_means(panelA)

    shade_A = [{"x0": 586, "x1": 589, "label": "Gaza war (Cast Lead)", "alpha": 0.25}]

    _plot_panel(
        panelA,
        out_pdf_path=str(figures_dir / "figure_1_a.pdf"),
        shade_blocks=shade_A,
        intifada_end_monthyear=541,
        xlabel_ticks=list(range(486, 623, 17)),
        xlabel_ticklabels=[str(v) for v in range(486, 623, 17)],
    )

    # Panel B: Excluding Gaza war months
    panelB = monthly.copy()

    # End of second Intifada missing
    panelB.loc[panelB["monthyear"] == 541, ["sum_vic", "sum_vic_pal"]] = np.nan

    # Gaza war months missing (587, 588)
    panelB.loc[panelB["monthyear"].isin([587, 588]), ["sum_vic", "sum_vic_pal"]] = np.nan

    panelB = _add_regime_means(panelB)

    shade_B = [
        {"x0": 505.5, "x1": 507.0, "label": "Defensive Shield", "alpha": 0.25},
        {"x0": 586.0, "x1": 589.0, "label": "Gaza war (Cast Lead)", "alpha": 0.25},
    ]

    _plot_panel(
        panelB,
        out_pdf_path=str(figures_dir / "figure_1_b.pdf"),
        shade_blocks=shade_B,
        intifada_end_monthyear=541,
        xlabel_ticks=list(range(486, 623, 17)),
        xlabel_ticklabels=[str(v) for v in range(486, 623, 17)],
    )

    print("Saved:", figures_dir / "figure_1_a.pdf")
    print("Saved:", figures_dir / "figure_1_b.pdf")


# ============================================================
# RUN
# ============================================================
df = load_replication_file(ROOT, DTA, DTA_FILENAME)
make_figure_1(df, figures_dir=FIGURES)


FileNotFoundError: Could not find replication_file1.dta.
Tried:
 - /content/dta/replication_file1.dta
 - /content/replication_file1.dta
And searched recursively under:
 - /content

Fix: either move the file into ./dta/ or set ROOT to the folder that contains it.

$$\textbf{ TABLA A1: Summary statistics for all vari-
ables used in the analysis}$$

In [ ]:
import pandas as pd
import numpy as np

# Charger les données (équivalent de: use "$dta/replication_file1.dta", clear)
df = pd.read_stata(DTA / "replication_file1.dta")

# Liste des variables utilisées dans tabstat
vars_tabstat = [
    "daily_woi",
    "daily_woi_nc",
    "any_conflict_news",
    "num_conflict_news",
    "length_conflict_news",
    "conflict_searches",
    "occurrence",
    "occurrence_pal",
    "victims_isr",
    "victims_pal",
    "lnvic",
    "lnvic_pal",
    "occurrence_target",
    "occurrence_non_target",
    "victims_target",
    "victims_non_target",
    "occurrence_all",
    "occurrence_fatal",
    "occurrence_non_fatal",
    "occurrence_hw",
    "occurrence_nhw",
    "occurrence_hpd",
    "occurrence_lpd",
    "fatal_victims",
    "non_fatal_victims",
    "victims_hw",
    "victims_nhw",
    "victims_hpd",
    "victims_lpd",
    "high_intensity",
    "lead_maj_events",
    "lead_disaster"
]

# Reproduire tabstat, statistics(N mean sd min max) column(statistics)
tabstat = (
    df[vars_tabstat]
    .agg(["count", "mean", "std", "min", "max"])
    .rename(index={
        "count": "N",
        "mean": "Mean",
        "std": "SD",
        "min": "Min",
        "max": "Max"
    })
    .T
)

# Afficher la table
tabstat


,N,Mean,SD,Min,Max
daily_woi,4071.0,0.886912,0.258157,0.216667,2.933333
daily_woi_nc,4071.0,0.867247,0.263514,0.216667,2.933333
any_conflict_news,4006.0,0.181727,0.385662,0.000000,1.000000
num_conflict_news,4006.0,0.327509,0.793908,0.000000,6.000000
length_conflict_news,4006.0,0.717216,2.543010,0.000000,36.000000
conflict_searches,2833.0,2.250398,0.670176,0.000000,4.771166
occurrence,4074.0,0.392244,0.488319,0.000000,1.000000
occurrence_pal,4074.0,0.067992,0.251762,0.000000,1.000000
victims_isr,4074.0,1.571183,7.423236,0.000000,356.000000
victims_pal,4074.0,0.158321,0.915384,0.000000,21.000000
